In [1]:
'''

File Name : CapacitatedVehicleRouting.py
Author : Mukaddes Altuntas
Created On : 20/04/2022
Version : Pyhton 3.7.5
Copyright (c) 2022 Mukaddes Altuntas. All rights reserved.
Description : Delivering daily orders to 103 stores from a depot 

'''


"""Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd
import numpy as np
import datetime
import folium
import json 
import requests

import warnings
warnings.filterwarnings('ignore')


pd.options.display.max_rows = 20000
pd.options.display.max_columns = 20000

import time

In [2]:
# Magazalar gunluk olarak siparis vermektedir.
siparis_gunu = 17

In [3]:
# Her magaza gunluk x palet urun siparis vermektedir. 
# 103 magaza siparisi, 1 depo = 0, 60 dummy depo siparisi = -18 (kamyon boşaltma icin negatif palet sayisi)
palet_bilgisi = pd.read_csv( str(siparis_gunu) + '_aralik_magaza_palet_bilgisi_nodes.csv')
palet_bilgisi.shape

(164, 8)

In [4]:
# Dummy depolar haric toplam palet sayisi
palet_bilgisi[palet_bilgisi['PALET_SAYISI']!= -18]['PALET_SAYISI'].sum()

413

In [5]:
# Bir noktadan diger noktaya gitmek icin gecen sure ve mesafe
dist_list = pd.read_csv(str(siparis_gunu) + '_aralik_distance_list_nodes.csv')
dist_list.head(2)

,Node_From,Node_To,Birim_From,Birim_To,Distance,Duration
0,"36.892053081353,34.7391309093348","36.892053081353,34.7391309093348",99999.0,99999.0,0.0,0.0
1,"36.892053081353,34.7391309093348","36.892053082352994,34.739130910334794",99999.0,99939.0,0.0,0.0


In [6]:
dist_list.rename(columns = {'Birim_To':'Birim'}, inplace = True)

In [7]:
dist_list['Birim'] = dist_list['Birim'].astype(float)
palet_bilgisi['Birim'] = palet_bilgisi['Birim'].astype(float)

In [8]:
dist_list = dist_list.merge(palet_bilgisi,on='Birim',how='left')

In [9]:
palet_bilgisi.shape, dist_list.shape

((164, 8), (26896, 13))

In [10]:
# Dummy depo palet sayisi = -18 (Doldurulan araci bosaltip tekrar sefere cikarmak icin arac kapasitesi kadar
#   
dist_list['PALET_SAYISI'].unique()

array([  0, -18,   5,   1,   8,   7,   4,   9,   6,   2,   3])

In [11]:
# 10 ve 18 paletlik araclar mevcuttur.
# Bazi magazalara fiziksel kosullardan dolayi sadece 10 paletlik arac gidebilmektedir. 
# Depoya ve diger magazalara 10 ve 18 paletlik arac gidebilmektedir. 
rota_opt = pd.read_excel('veriler/sok_palet_magaza.xlsx', sheet_name = 'Mağaza Araç Tipi')
rota_opt.head()

,Depo No,Depo Adý,Mađaza No,Mađaza Adý,Araç Tipi,Yükleme Tip
0,3625,MERSÝN DAĐITIM MERKEZÝ,10031,MERSÝN YENÝ HÜRRÝYET,18 PLT,TPL
1,3625,MERSÝN DAĐITIM MERKEZÝ,10089,SÝNOP AYANCIK BEŢÝKTAŢ,18 PLT,TPL
2,3625,MERSÝN DAĐITIM MERKEZÝ,10115,MERSÝN YENÝ TÖMÜK,18 PLT,TPL
3,3625,MERSÝN DAĐITIM MERKEZÝ,10180,ERDEMLÝ YAVUZ SULTAN SELÝ,18 PLT,TPL
4,3625,MERSÝN DAĐITIM MERKEZÝ,10237,ERDEMLÝ PLAJ YOLU,18 PLT,TPL


In [12]:
# veriyi duzenleme
rota_opt.rename(columns={'Birim Adý':'Birim Adı', 'Depo Adý':'Depo Adı', 'Mađaza No':'Magaza No', 'Mađaza Adý':'Magaza Adı'},inplace=True)

In [13]:
# veriyi duzenleme
cols  = ['Depo Adı','Magaza Adı']
for column in cols:
    rota_opt[column]=rota_opt[column].str.replace('ý','ı')
    rota_opt[column]=rota_opt[column].str.replace('Ý','İ')
    rota_opt[column]=rota_opt[column].str.replace('Þ','Ş')
    rota_opt[column]=rota_opt[column].str.replace('Ð','Ğ')

In [14]:
rota_opt['Araç Tipi'] = rota_opt['Araç Tipi'].str.replace(' PLT', '')

In [15]:
# 10 ve 18 paletlik iki arac tipi mevcuttur
rota_opt['Araç Tipi'].unique()

array(['18', '10'], dtype=object)

In [16]:
rota_opt['Araç Tipi'] = rota_opt['Araç Tipi'].astype(int)

In [17]:
rota_opt = rota_opt.sort_values('Araç Tipi', ascending = True)
rota_opt.head(2)

,Depo No,Depo Adı,Magaza No,Magaza Adı,Araç Tipi,Yükleme Tip
255,3625,MERSİN DAĐITIM MERKEZİ,8312,MERSİN DEVLET HASTANESİ,10,TPL
214,3625,MERSİN DAĐITIM MERKEZİ,6768,MERSİN KİREMİTHANE,10,TPL


In [18]:
# 10 paletlik aracin km basina yakma orani 0.23
# 18 paletlik aracin km basina yakma orani 0.3
rota_opt.loc[rota_opt['Araç Tipi'] == 10, 'Yakit_TL'] = 0.23
rota_opt.loc[rota_opt['Araç Tipi'] == 18, 'Yakit_TL'] = 0.3

In [19]:
# Aylik kiralanan a10 ve 18 paletlik araclarin aylik gunluk kiralama bedeli 
rota_opt.loc[rota_opt['Araç Tipi'] == 18, 'Amortisman_TL'] = round(9299 / 30)
rota_opt.loc[rota_opt['Araç Tipi'] == 10, 'Amortisman_TL'] = round(7531 / 30)

In [20]:
# Rotalama algoritmasi girdileri liste olarak kabul etmektedir.
capacities = list(rota_opt['Araç Tipi'])
yakit =  list(rota_opt['Yakit_TL'])
amortisman =  list(rota_opt['Amortisman_TL'])

In [21]:
node_and_distances = dist_list.groupby('Node_From')['Distance'].apply(list).reset_index()
node_and_distances.shape

(164, 2)

In [22]:
# Bir koordinattan diger koordinatlara olan uzakliklar
node_and_distances.head()

,Node_From,Distance
0,"36.337427,33.402297","[162.506, 162.506, 162.506, 162.506, 162.506, ..."
1,"36.641446,33.450411","[176.53779999999998, 176.53779999999998, 176.5..."
2,"36.643121,33.436608","[176.9328, 176.9328, 176.9328, 176.9328, 176.9..."
3,"36.643721,33.442925","[176.76729999999998, 176.76729999999998, 176.7..."
4,"36.644405,33.438601","[177.1569, 177.1569, 177.1569, 177.1569, 177.1..."


In [23]:
# Bir koordinattan diger koordinatlara gitmek icin gecen sure
node_and_distances_duration = dist_list.groupby('Node_From')['Duration'].apply(list).reset_index()
node_and_distances_duration.shape

(164, 2)

In [24]:
node_and_distances = node_and_distances.merge(node_and_distances_duration,on='Node_From',how='left')

In [25]:
dist_list = dist_list.drop_duplicates('Birim', keep = 'first')

In [26]:
# Bir noktadan diger noktalara olan uzakliklar ve sure bilgisi, magaza kodu, 
# magazanin talep ettigi siparis bilgisi, magaza adi, magazaya giden arac tipi 
node_and_distances = node_and_distances.merge(dist_list[['Node_To','Birim','PALET_SAYISI','MAĞAZA ADI','giden_arac_kapasite']], how = 'left', left_on = 'Node_From', right_on = 'Node_To')
node_and_distances.head(2)

,Node_From,Distance,Duration,Node_To,Birim,PALET_SAYISI,MAĞAZA ADI,giden_arac_kapasite
0,"36.337427,33.402297","[162.506, 162.506, 162.506, 162.506, 162.506, ...","[142.97833333333335, 142.97833333333335, 142.9...","36.337427,33.402297",8384.0,7,ŞOK-MERSİN GÜLNAR,18.0
1,"36.641446,33.450411","[176.53779999999998, 176.53779999999998, 176.5...","[145.75, 145.75, 145.75, 145.75, 145.75, 145.7...","36.641446,33.450411",11401.0,5,ŞOK-MERSİN MUT YAZALANI,18.0


In [27]:
node_and_distances.PALET_SAYISI.unique()

array([  7,   5,   6,   4,   1,   9,   8,   3,   2,   0, -18])

In [28]:
node_and_distances.drop(columns= 'Node_To', inplace = True)

In [29]:
# Depo ve dummy depolar ilk 61 satirda, magazalar sonraki satirlarda olacak sekilde siralama yapilmistir
node_and_distances['ind'] = node_and_distances.Distance.apply(lambda x: [i for i in range(len(x)) if x[i] == 0])
node_and_distances.head(2)

,Node_From,Distance,Duration,Birim,PALET_SAYISI,MAĞAZA ADI,giden_arac_kapasite,ind
0,"36.337427,33.402297","[162.506, 162.506, 162.506, 162.506, 162.506, ...","[142.97833333333335, 142.97833333333335, 142.9...",8384.0,7,ŞOK-MERSİN GÜLNAR,18.0,[83]
1,"36.641446,33.450411","[176.53779999999998, 176.53779999999998, 176.5...","[145.75, 145.75, 145.75, 145.75, 145.75, 145.7...",11401.0,5,ŞOK-MERSİN MUT YAZALANI,18.0,[98]


In [30]:
node_and_distances= node_and_distances.sort_values(by = ['ind'], ascending = True)

In [31]:
node_and_distances['ind2'] = node_and_distances['ind'].apply(lambda x: x[0])

In [32]:
node_and_distances = node_and_distances.sort_values(by = ['ind2','Birim'], ascending = [True,False])
node_and_distances.head()

,Node_From,Distance,Duration,Birim,PALET_SAYISI,MAĞAZA ADI,giden_arac_kapasite,ind,ind2
71,"36.892053081353,34.7391309093348","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99999.0,0,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0
131,"36.89205314135279,34.73913096933459","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99998.0,-18,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0
130,"36.89205314035279,34.73913096833459","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99997.0,-18,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0
129,"36.8920531393528,34.739130967334596","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99996.0,-18,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0
128,"36.8920531383528,34.7391309663346","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99995.0,-18,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0


In [33]:
node_and_distances= node_and_distances.reset_index(drop = True)

In [34]:
len(node_and_distances)

164

In [35]:
dists = []

for node in range(len(node_and_distances)):
    dists.append(node_and_distances.loc[node]['Distance'])

node_and_distances

,Node_From,Distance,Duration,Birim,PALET_SAYISI,MAĞAZA ADI,giden_arac_kapasite,ind,ind2
0,"36.892053081353,34.7391309093348","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99999.0,0,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0
1,"36.89205314135279,34.73913096933459","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99998.0,-18,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0
2,"36.89205314035279,34.73913096833459","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99997.0,-18,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0
3,"36.8920531393528,34.739130967334596","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99996.0,-18,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0
4,"36.8920531383528,34.7391309663346","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99995.0,-18,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0
5,"36.892053137352804,34.7391309653346","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99994.0,-18,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0
6,"36.89205313635281,34.739130964334606","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99993.0,-18,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0
7,"36.89205313535281,34.73913096333461","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99992.0,-18,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0
8,"36.892053134352814,34.73913096233461","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99991.0,-18,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0
9,"36.89205313335282,34.73913096133462","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99990.0,-18,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0


In [36]:
np.max(node_and_distances.loc[1, 'Distance'])

183.4658

In [37]:
duration = []

for node in range(len(node_and_distances)):
    duration.append(node_and_distances.loc[node]['Duration'])

node_and_distances.head(2)

,Node_From,Distance,Duration,Birim,PALET_SAYISI,MAĞAZA ADI,giden_arac_kapasite,ind,ind2
0,"36.892053081353,34.7391309093348","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99999.0,0,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0
1,"36.89205314135279,34.73913096933459","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99998.0,-18,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0


In [38]:
node_and_distances.shape

(164, 9)

In [39]:
palet_bilgisi.PALET_SAYISI.unique()

array([  0, -18,   5,   1,   8,   7,   4,   9,   6,   2,   3])

In [40]:
node_and_distances.PALET_SAYISI.unique()

array([  0, -18,   5,   1,   8,   7,   4,   9,   6,   2,   3])

In [41]:
node_and_distances.reset_index(drop = True, inplace = True)

In [42]:
palet_talep = list(node_and_distances.PALET_SAYISI)

In [43]:
arac_tipi = list(node_and_distances['giden_arac_kapasite'])

In [44]:
# Depo 8.00-17.00, magazalar 8.00-20.00 arasi aciktir
# Ilk depo icin time window (0,0)
# Dummy depolar icin time window (0,540)
# Magazalar icin time window (0,720)
time_windows = []
time_windows.append((0,0))
for i in range(1,61):
    time_windows.append((0,540))
for i in range(61, len(palet_talep)):
    time_windows.append((0,720))

In [45]:
len(palet_talep)

164

In [46]:
len(arac_tipi)

164

In [47]:
time_windows

[(0, 0),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 540),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 (0, 720),
 

In [48]:
palet_talep

[0,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 -18,
 5,
 1,
 8,
 7,
 4,
 9,
 4,
 5,
 6,
 1,
 4,
 4,
 5,
 2,
 5,
 6,
 1,
 5,
 7,
 7,
 1,
 1,
 7,
 1,
 7,
 1,
 1,
 1,
 1,
 7,
 6,
 6,
 5,
 6,
 6,
 6,
 5,
 5,
 5,
 5,
 5,
 1,
 1,
 1,
 6,
 1,
 7,
 6,
 1,
 1,
 5,
 5,
 6,
 1,
 1,
 1,
 1,
 1,
 1,
 8,
 6,
 1,
 1,
 6,
 1,
 6,
 2,
 5,
 1,
 6,
 7,
 6,
 6,
 1,
 1,
 3,
 1,
 1,
 8,
 5,
 4,
 7,
 7,
 8,
 5,
 6,
 5,
 4,
 5,
 5,
 5,
 5,
 5,
 7,
 6,
 6,
 4,
 1,
 1,
 1,
 1,
 1,
 1]

In [49]:
len(palet_talep)

164

In [50]:
# Eldeki veriler dogrultusunda servis suresi ilk palet icin 13 dk,
# sonraki paletler icin 6 dk olacak sekilde hesaplanmistir.
# Sonraki adimda bu bilgi kullanilacaktir. 
services_duration = []
for node in range(len(palet_talep)):
    services_duration.append(abs(palet_talep[node]))
services_duration

[0,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 5,
 1,
 8,
 7,
 4,
 9,
 4,
 5,
 6,
 1,
 4,
 4,
 5,
 2,
 5,
 6,
 1,
 5,
 7,
 7,
 1,
 1,
 7,
 1,
 7,
 1,
 1,
 1,
 1,
 7,
 6,
 6,
 5,
 6,
 6,
 6,
 5,
 5,
 5,
 5,
 5,
 1,
 1,
 1,
 6,
 1,
 7,
 6,
 1,
 1,
 5,
 5,
 6,
 1,
 1,
 1,
 1,
 1,
 1,
 8,
 6,
 1,
 1,
 6,
 1,
 6,
 2,
 5,
 1,
 6,
 7,
 6,
 6,
 1,
 1,
 3,
 1,
 1,
 8,
 5,
 4,
 7,
 7,
 8,
 5,
 6,
 5,
 4,
 5,
 5,
 5,
 5,
 5,
 7,
 6,
 6,
 4,
 1,
 1,
 1,
 1,
 1,
 1]

In [51]:
len(capacities), len(palet_talep), len(dists), len(yakit)

(341, 164, 164, 341)

In [52]:
np.sum(palet_talep) + 60*18

413

In [53]:
np.sum(capacities)

5986

In [54]:
len(palet_talep),len(capacities),len(duration), len(time_windows), len(services_duration)


(164, 341, 164, 164, 164)

In [55]:
len(capacities)

341

In [56]:
# Bir magazaya tum araclar gidebiliyorsa tum araclarin indexlerini al
# Ortoolsta ilk deger -1 olarak verilmeli
visiting_vehicles_0 = [-1]
for i in range(0,len(capacities)):
    visiting_vehicles_0.append(i)
visiting_vehicles_0

[-1,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 

In [57]:
# Bir magazaya 10 paletlik arac gidebiliyorsa 10 paletlik araclarin indexlerini al
visiting_vehicles_10 = [-1]
for i in range(0,len(capacities)):
    if capacities[i] == 10:
        visiting_vehicles_10.append(i)
visiting_vehicles_10

[-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [58]:
len(visiting_vehicles_10)

20

In [59]:
# Bir magazaya 18 paletlik arac gidebiliyorsa 18 paletlik araclarin indexlerini al
visiting_vehicles_18 = [-1]
for i in range(0,len(capacities)):
    if capacities[i] == 18:
        visiting_vehicles_18.append(i)
visiting_vehicles_18

[-1,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 1

In [60]:
len(visiting_vehicles_18)

323

In [61]:
# depo = 0 ve 60 tane dummy depo (coklu sefer icin)
depolar = []
for i in range(0,61):
    depolar.append(i)
    
depolar

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60]

In [62]:
# nearest neighbor a benzer bir heuristic ile kodlanana baslangic cozumu algoritmaya verilecek.
initial_solution = pd.read_csv('veriler/initial_solution.csv')
initial_solution.head(2)

,arac_id,capacities,routes,birim_list,magaza_list,palet_list,dist_list,sefer_id,Birim_To,max_distance
0,0,18.0,0,10238,ŞOK-NİĞDE ŞAFAK,7,0,0,10238,370.0
1,0,18.0,0,7790,ŞOK-NİĞDE FERTEK,5,0,0,7790,370.0


In [63]:
node_and_distances2 = node_and_distances.reset_index()
node_and_distances2.head(2)

,index,Node_From,Distance,Duration,Birim,PALET_SAYISI,MAĞAZA ADI,giden_arac_kapasite,ind,ind2
0,0,"36.892053081353,34.7391309093348","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99999.0,0,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0
1,1,"36.89205314135279,34.73913096933459","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99998.0,-18,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0


In [64]:
initial_solution = initial_solution.merge(node_and_distances2[['index','Birim']], left_on = 'birim_list', right_on = 'Birim', how = 'left')
initial_solution.head(2)

,arac_id,capacities,routes,birim_list,magaza_list,palet_list,dist_list,sefer_id,Birim_To,max_distance,index,Birim
0,0,18.0,0,10238,ŞOK-NİĞDE ŞAFAK,7,0,0,10238,370.0,154,10238.0
1,0,18.0,0,7790,ŞOK-NİĞDE FERTEK,5,0,0,7790,370.0,149,7790.0


In [65]:
atanan_seferler = initial_solution.groupby(['arac_id','capacities'])['index'].apply(list).reset_index()
atanan_seferler.head()

,arac_id,capacities,index
0,0,18.0,"[154, 149, 150]"
1,1,18.0,"[144, 151]"
2,2,18.0,"[143, 152, 145]"
3,3,18.0,"[153, 147, 155]"
4,4,18.0,"[148, 146, 113, 119, 159]"


In [66]:
atanan_seferler10 = atanan_seferler[atanan_seferler['capacities'] == 10]
atanan_seferler18 = atanan_seferler[atanan_seferler['capacities'] == 18]
atanan_seferler10 = atanan_seferler10.reset_index(drop = True)
atanan_seferler18 = atanan_seferler18.reset_index(drop = True)

In [67]:
counter_10 = 0
counter_18 = 0

for i in range(len(capacities)):
    if capacities[i] == 10:
        counter_10 += 1
    else:
        counter_18 += 1

In [68]:
counter_10, counter_18

(19, 322)

In [69]:
seferler10 = []
for i in range(len(atanan_seferler10)):
    print(i)
    print(atanan_seferler10.loc[i,'index'])
    seferler10.append(atanan_seferler10.loc[i,'index'])
for i in range(counter_10-len(seferler10)):
     seferler10.append([])
    

0
[69, 136]


In [70]:
seferler18 = []
for i in range(len(atanan_seferler18)):
    print(i)
    print(atanan_seferler18.loc[i,'index'])
    seferler18.append(atanan_seferler18.loc[i,'index'])
for i in range(counter_18-len(seferler18)):
     seferler18.append([])
    

0
[154, 149, 150]
1
[144, 151]
2
[143, 152, 145]
3
[153, 147, 155]
4
[148, 146, 113, 119, 159]
5
[97, 99, 96]
6
[95, 98, 83]
7
[87, 141, 71, 76, 81]
8
[73, 128, 107]
9
[124, 108, 94]
10
[90, 100, 92]
11
[101, 72, 64]
12
[133, 93, 104, 130]
13
[139, 85]
14
[66, 78]
15
[105, 142, 68]
16
[61, 63]
17
[79, 91, 127, 123]
18
[80, 106, 134, 103, 75, 89, 74]
19
[65, 77, 67, 137, 82, 129, 109, 88, 84]
20
[86, 62, 138, 110, 70, 125, 132, 111]
21
[122, 163, 157, 115, 114, 161, 118, 112, 158, 162]
22
[102, 126, 135, 140]
23
[131, 116, 160, 117, 156]
24
[120, 121]


In [71]:
seferler_tum = seferler10 + seferler18
seferler_tum

[[69, 136],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [154, 149, 150],
 [144, 151],
 [143, 152, 145],
 [153, 147, 155],
 [148, 146, 113, 119, 159],
 [97, 99, 96],
 [95, 98, 83],
 [87, 141, 71, 76, 81],
 [73, 128, 107],
 [124, 108, 94],
 [90, 100, 92],
 [101, 72, 64],
 [133, 93, 104, 130],
 [139, 85],
 [66, 78],
 [105, 142, 68],
 [61, 63],
 [79, 91, 127, 123],
 [80, 106, 134, 103, 75, 89, 74],
 [65, 77, 67, 137, 82, 129, 109, 88, 84],
 [86, 62, 138, 110, 70, 125, 132, 111],
 [122, 163, 157, 115, 114, 161, 118, 112, 158, 162],
 [102, 126, 135, 140],
 [131, 116, 160, 117, 156],
 [120, 121],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 

In [72]:
# ## RUNNED CODE START
"""Capacitated Vehicle Routing Problem with Time Windows (CVRPTW).
   This is a sample using the routing library python wrapper to solve a CVRPTW
   problem.
   A description of the problem can be found here:
   http://en.wikipedia.org/wiki/Vehicle_routing_problem.
   Distances are in meters and time in minutes.
"""

# [START import]
from functools import partial
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
# [END import]

def create_data_model():
    """Stores the data for the problem for 130 nodes."""
    data = {}
    data['distance_matrix'] = dists
    data['demands'] = palet_talep
    data['vehicle_capacities'] = capacities
    data['num_vehicles'] = len(capacities)
    data['num_locations'] = len(palet_talep)
    data['depot'] = 0
    data['vehicle_costs'] = yakit
    data['amortization'] = amortisman
    data['initial_routes'] = seferler_tum

    
# araçların aynı hızda gittiğini varsayarak time matrix yerine dist matrixi de kullanabiliriz.
    data['time_matrix'] = duration
    data['time_windows'] = time_windows
    data['service_time'] = services_duration
    
    
    return data

#######################
# Problem Constraints #
#######################

def create_distance_evaluator(data):
    """Creates callback to return distance between points."""
    _distances = data['distance_matrix']


    def distance_evaluator(manager, from_node, to_node):
        """Returns the manhattan distance between the two nodes"""
        return data['distance_matrix'][manager.IndexToNode(from_node)][manager.IndexToNode(to_node)]

    return distance_evaluator


def create_demand_evaluator(data):
    """Creates callback to get demands at each location."""
    _demands = data['demands']

    def demand_evaluator(manager, node):
        """Returns the demand of the current node"""
        return _demands[manager.IndexToNode(node)]

    return demand_evaluator


def add_capacity_constraints(routing, manager, data, demand_evaluator_index):
    """Adds capacity constraint"""
    capacity = 'Capacity'
    routing.AddDimensionWithVehicleCapacity(
        demand_evaluator_index,
        0,  # null capacity slack
        data['vehicle_capacities'],
        True,  # start cumul to zero
        capacity)
        
    # Add Slack for reseting to zero unload depot nodes.
    # e.g. vehicle with load 10/15 arrives at node 1 (depot unload)
    # so we have CumulVar = 10(current load) + -15(unload) + 5(slack) = 0.
    capacity_dimension = routing.GetDimensionOrDie(capacity)
    # Allow to drop reloading nodes with zero cost.
    for node in range(1,60):
        node_index = manager.NodeToIndex(node)
        routing.AddDisjunction([node_index], 0)

    # Allow to drop regular node with a cost.
    for node in range(61, len(data['demands'])):
        node_index = manager.NodeToIndex(node)
        capacity_dimension.SlackVar(node_index).SetValue(0)
#         drop the node with 100000 penalty points
#         routing.AddDisjunction([node_index], 100_000)
    
def add_hiring_cost_constraints(routing, penalty):
    for vehicle_id in range(data['num_vehicles']):
        routing.SetFixedCostOfVehicle(penalty[vehicle_id], vehicle_id)

def create_time_evaluator(data):
    """Creates callback to get total times between locations."""

    def service_time(data, node):
        """Gets the service time for the specified location."""
#      13 dakika ilk palet icin, sonraki paletler icin 6 dk 
        return abs((13 + ((data['demands'][node]-1) * 6 )))
    def travel_time(data, from_node, to_node):
        """Gets the travel times between two locations."""

        return data['time_matrix'][from_node][to_node]
        

    _total_time = {}
    # precompute total time to have time callback in O(1)
    for from_node in range(data['num_locations']):
        _total_time[from_node] = {}
        for to_node in range(data['num_locations']):
            if from_node == to_node:
                _total_time[from_node][to_node] = 0
            else:
                _total_time[from_node][to_node] = int(
                    service_time(data, from_node) + travel_time(
                        data, from_node, to_node))
#                 print('totallll',  _total_time[from_node][to_node] )

    def time_evaluator(manager, from_node, to_node):
        """Returns the total time between the two nodes"""
#         print('totalde' ,  _total_time[manager.IndexToNode(from_node)][manager.IndexToNode(
#             to_node)])
        return _total_time[manager.IndexToNode(from_node)][manager.IndexToNode(to_node)]

    return time_evaluator


def add_time_window_constraints(routing, manager, data, time_evaluator_index):
    """Add Global Span constraint"""
    time = 'Time'
    horizon = 720
    routing.AddDimension(
        time_evaluator_index,
        horizon,  # allow waiting time
        horizon,  # maximum time per vehicle
        False,  # don't force start cumul to zero since we are giving TW to start nodes
        time)
    time_dimension = routing.GetDimensionOrDie(time)
    # Add time window constraints for each location except depot
    # and 'copy' the slack var in the solution object (aka Assignment) to print it
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == 0:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
        routing.AddToAssignment(time_dimension.SlackVar(index))
    # Add time window constraints for each vehicle start node
    # and 'copy' the slack var in the solution object (aka Assignment) to print it
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(data['time_windows'][0][0],
                                                data['time_windows'][0][1])
        routing.AddToAssignment(time_dimension.SlackVar(index))
        # Warning: Slack var is not defined for vehicle's end node
        #routing.AddToAssignment(time_dimension.SlackVar(self.routing.End(vehicle_id)))


# [START solution_printer]
def print_solution(manager, routing, assignment, data):  # pylint:disable=too-many-locals
    """Prints assignment on console"""
    print(f'Objective: {assignment.ObjectiveValue()}')
    time_dimension = routing.GetDimensionOrDie('Time')
    capacity_dimension = routing.GetDimensionOrDie('Capacity')
    total_distance = 0
    total_load = 0
    total_time = 0
    dist_dict = {}
    
#     60 depots are used as dummy depots to make multi-trips
    nodes_to_ignore = []
    for i in range(1,61):
        nodes_to_ignore.append(i)

    for vehicle_id in range(manager.GetNumberOfVehicles()):
        dists=[]
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {} ({} plt):\n'.format(vehicle_id,capacities[vehicle_id] )
        distance = 0
        total_load_var = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            load_var = capacity_dimension.CumulVar(index)
            time_var = time_dimension.CumulVar(index)
            slack_var = time_dimension.SlackVar(index)

            if manager.IndexToNode(index) in nodes_to_ignore:
                node_index = manager.IndexToNode(index)
                plan_output += '\n\n#DEPOT{0}#'.format( manager.IndexToNode(index))
                total_load_var += assignment.Value(capacity_dimension.CumulVar(previous_index))
            else:
                plan_output += '#{0}#'.format(manager.IndexToNode(index))
              
            previous_index = index
            previous_index = manager.IndexToNode(index)
            index = assignment.Value(routing.NextVar(index))
            
            distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
            load_var = capacity_dimension.CumulVar(index)
            time_var = time_dimension.CumulVar(index)
            slack_var = time_dimension.SlackVar(index)
            plan_output += ' CumLoadNext({0})'.format(assignment.Value(capacity_dimension.CumulVar(index)))

            plan_output += ' CumDist({0}) Dist({1})'.format(distance-1000, routing.GetArcCostForVehicle(previous_index, index,
                                                     vehicle_id))
            
            plan_output += ' CumTime({0}) ->'.format(assignment.Min(time_var))
            dists.append(node_index)
        dists.append(0)
        dist_dict[vehicle_id] = dists
    
        load_var = capacity_dimension.CumulVar(index)
        time_var = time_dimension.CumulVar(index)
        slack_var = time_dimension.SlackVar(index)
        total_load_var += assignment.Value(capacity_dimension.CumulVar(previous_index))


        plan_output += ' {0} Load({1}) Time({2})\n'.format(
            manager.IndexToNode(index),
            assignment.Value(load_var),
            assignment.Min(time_var))
        plan_output += 'Distance of the route: {0} km\n'.format(distance)
        plan_output += 'Load of the route: {}\n'.format(
            total_load_var)
        plan_output += 'Time of the route: {} hours\n'.format(
            assignment.Min(time_var)/60)
        print(plan_output)
        total_distance += distance
        total_load += total_load_var
        total_time += assignment.Min(time_var)
    print('Total Distance of all routes: {0} km'.format(total_distance))
    print('Total Load of all routes: {}'.format(total_load))
    print('Total Time of all routes: {0} hours'.format(total_time/60))
    return dist_dict, data
    # [END solution_printer]


def main():
    """Solve the Capacitated VRP with time windows."""
    # Instantiate the data problem.
    # [START data]
    data = create_data_model()
    # [END data]

    # Create the routing index manager.
    # [START index_manager]
    manager = pywrapcp.RoutingIndexManager(data['num_locations'],
                                           data['num_vehicles'], data['depot'])
    # [END index_manager]

    # Create Routing Model.
    # [START routing_model]
    routing = pywrapcp.RoutingModel(manager)
    # [END routing_model]

#     Different capacitated vehicles go to the same length distance 
    callback_indices = []
    for vehicle_idx in range(data['num_vehicles']):
        def vehicle_callback(from_index, to_index, i=vehicle_idx):
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
#             Iki node arasi mesafenin 60 km az olmasi icin kisit Verildi / Verilmedi
            if (from_index in depolar) or (to_index in depolar):
                return data['vehicle_costs'][i] * data['distance_matrix'][from_node][to_node]
#                 return data['distance_matrix'][from_node][to_node]
#                 return 0
            else:
                if data['distance_matrix'][from_node][to_node] < 60:
                    return data['vehicle_costs'][i] * data['distance_matrix'][from_node][to_node]
#                     return data['distance_matrix'][from_node][to_node]
                else:
                    return data['vehicle_costs'][i] * data['distance_matrix'][from_node][to_node]
#                     return 400

        callback_index = routing.RegisterTransitCallback(vehicle_callback)

        routing.SetArcCostEvaluatorOfVehicle(callback_index, vehicle_idx)

        callback_indices.append(callback_index)

    routing.AddDimensionWithVehicleTransits(
        callback_indices,
        0,
        50000,
        True,
        'Distance')    


    for store_idx in range(len(data['demands'])):
        if store_idx in range(0,60):
#             print(store_idx)
            index = manager.NodeToIndex(store_idx)
            routing.SetAllowedVehiclesForIndex(visiting_vehicles_0, index) # remove vehicle 0
        elif (store_idx >= 61):
            if arac_tipi[store_idx] == 10:
                index = manager.NodeToIndex(store_idx)
                routing.SetAllowedVehiclesForIndex(visiting_vehicles_10, index) # remove vehicle 0    
            else:
                index = manager.NodeToIndex(store_idx)
                routing.SetAllowedVehiclesForIndex(visiting_vehicles_0, index) # remove vehicle 0    

    demand_evaluator_index = routing.RegisterUnaryTransitCallback(
        partial(create_demand_evaluator(data), manager))
    add_capacity_constraints(routing, manager, data, demand_evaluator_index)
    # [END capacity_constraint]

    # Add Time Window constraint.
    # [START time_constraint]
    time_evaluator_index = routing.RegisterTransitCallback(
        partial(create_time_evaluator(data), manager))
    add_time_window_constraints(routing, manager, data, time_evaluator_index)
    # [END time_constraint]
    
    # Add hiring cost constraint.
    for vehicle_idx in range(data['num_vehicles']):
#         print(data['amortization'][vehicle_idx])
       routing.SetFixedCostOfVehicle(int(data['amortization'][vehicle_idx]), vehicle_idx)
    
    
    initial_solution = routing.ReadAssignmentFromRoutes(data['initial_routes'],
                                                        True)
#     print('Initial solution:')
#     print_solution(data, manager, routing, initial_solution)

    # Set default search parameters.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()

    


    # Setting first solution heuristic (cheapest addition).
    # [START parameters]
#     search_parameters = pywrapcp.DefaultRoutingSearchParameters()
#     search_parameters.first_solution_strategy = (
#         routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.TABU_SEARCH)
#     search_parameters.time_limit.FromSeconds(2)
    search_parameters.time_limit.seconds = 160

    search_parameters.log_search = True
    # [END parameters]
    
    
    # Solve the problem.
    solution = routing.SolveFromAssignmentWithParameters(
        initial_solution, search_parameters)

#     # Solve the problem.
#     # [START solve]
#     solution = routing.SolveWithParameters(search_parameters)
#     # [END solve]

    # Print solution on console.
    # [START print_solution]
    if solution:
        dist_dict, data = print_solution(manager, routing, solution, data)
        return dist_dict, data
    else:
        print('Rota olusturulamadi.')
        return None, None
if __name__ == '__main__':
    dist_dict, data = main()
    dist_dict, data
# ## RUNNED CODE END

Objective: 5856
Route for vehicle 0 (10 plt):
#0# CumLoadNext(0) CumDist(-708) Dist(292) CumTime(134) ->#143# CumLoadNext(7) CumDist(-672) Dist(36) CumTime(293) ->#117# CumLoadNext(8) CumDist(-669) Dist(3) CumTime(323) -> 0 Load(8) Time(323)
Distance of the route: 331 km
Load of the route: 7
Time of the route: 5.383333333333334 hours

Route for vehicle 1 (10 plt):
#0# CumLoadNext(0) CumDist(-1000) Dist(0) CumTime(0) -> 0 Load(0) Time(0)
Distance of the route: 0 km
Load of the route: 0
Time of the route: 0.0 hours

Route for vehicle 2 (10 plt):
#0# CumLoadNext(0) CumDist(-1000) Dist(0) CumTime(0) -> 0 Load(0) Time(0)
Distance of the route: 0 km
Load of the route: 0
Time of the route: 0.0 hours

Route for vehicle 3 (10 plt):
#0# CumLoadNext(0) CumDist(-1000) Dist(0) CumTime(0) -> 0 Load(0) Time(0)
Distance of the route: 0 km
Load of the route: 0
Time of the route: 0.0 hours

Route for vehicle 4 (10 plt):
#0# CumLoadNext(0) CumDist(-1000) Dist(0) CumTime(0) -> 0 Load(0) Time(0)
Distance o

In [73]:
len(palet_bilgisi)

164

In [74]:
dist_dict

{0: [0, 143, 117, 0],
 1: [0, 0],
 2: [0, 0],
 3: [0, 0],
 4: [0, 0],
 5: [0, 0],
 6: [0, 0],
 7: [0, 0],
 8: [0, 0],
 9: [0, 0],
 10: [0, 0],
 11: [0, 0],
 12: [0, 0],
 13: [0, 0],
 14: [0, 0],
 15: [0, 0],
 16: [0, 0],
 17: [0, 0],
 18: [0, 88, 69, 136, 0],
 19: [0, 0],
 20: [0, 0],
 21: [0, 0],
 22: [0, 0],
 23: [0, 0],
 24: [0, 0],
 25: [0, 0],
 26: [0, 0],
 27: [0, 0],
 28: [0, 0],
 29: [0, 0],
 30: [0, 0],
 31: [0, 0],
 32: [0, 0],
 33: [0, 0],
 34: [0, 0],
 35: [0, 0],
 36: [0, 0],
 37: [0, 0],
 38: [0, 0],
 39: [0, 0],
 40: [0, 0],
 41: [0, 0],
 42: [0, 0],
 43: [0, 0],
 44: [0, 0],
 45: [0, 0],
 46: [0, 0],
 47: [0, 0],
 48: [0, 0],
 49: [0, 0],
 50: [0, 0],
 51: [0, 0],
 52: [0, 0],
 53: [0, 0],
 54: [0, 0],
 55: [0, 0],
 56: [0, 0],
 57: [0, 0],
 58: [0, 0],
 59: [0, 0],
 60: [0, 0],
 61: [0, 0],
 62: [0, 0],
 63: [0, 0],
 64: [0, 0],
 65: [0, 0],
 66: [0, 0],
 67: [0, 0],
 68: [0, 0],
 69: [0, 0],
 70: [0, 0],
 71: [0, 0],
 72: [0, 0],
 73: [0, 0],
 74: [0, 0],
 75: [0, 0],

In [75]:
arac_tipi
for i in range(len(arac_tipi)):
    if arac_tipi[i] == 10:
        print(i, arac_tipi[i])

69 10.0
136 10.0


In [76]:
mesafe = {}
routes = {}

total_sum = 0
for vec in dist_dict.keys():
    print('*******************************')
    print('Vehicle', vec)
    print('----------')
    total_dist_meter = 0
    points = []
    for i in range(len(dist_dict[vec])-1):
        points.append(dist_dict[vec][i])
        
        print(dist_dict[vec][i], '-->', dist_dict[vec][i+1], data['distance_matrix'][dist_dict[vec][i]][ dist_dict[vec][i+1]])
        total_dist_meter += data['distance_matrix'][dist_dict[vec][i]][ dist_dict[vec][i+1]]
    total_sum += total_dist_meter    
    mesafe[vec] = total_dist_meter
    points.append(0)
    routes[vec] = points
print('*******************************')
print('Total Dist:', total_sum)
print('*******************************')

*******************************
Vehicle 0
----------
0 --> 143 181.8487
143 --> 117 160.65460000000002
117 --> 0 17.3322
*******************************
Vehicle 1
----------
0 --> 0 0.0
*******************************
Vehicle 2
----------
0 --> 0 0.0
*******************************
Vehicle 3
----------
0 --> 0 0.0
*******************************
Vehicle 4
----------
0 --> 0 0.0
*******************************
Vehicle 5
----------
0 --> 0 0.0
*******************************
Vehicle 6
----------
0 --> 0 0.0
*******************************
Vehicle 7
----------
0 --> 0 0.0
*******************************
Vehicle 8
----------
0 --> 0 0.0
*******************************
Vehicle 9
----------
0 --> 0 0.0
*******************************
Vehicle 10
----------
0 --> 0 0.0
*******************************
Vehicle 11
----------
0 --> 0 0.0
*******************************
Vehicle 12
----------
0 --> 0 0.0
*******************************
Vehicle 13
----------
0 --> 0 0.0
******************************

In [77]:
sure = {}
routes = {}
gidis_bilgisi = pd.DataFrame(columns={'arac_index', 'index_from', 'index_to', 'mesafe_km', 'zaman_dk','servis_suresi'})


for vec in dist_dict.keys():
    
    total_sum_min = 0
    print('*******************************')
    print('Vehicle', vec)
    print('----------')
    total_dur_min = 0
    points = []
    for i in range(len(dist_dict[vec])-1):
        gidis_bilgisi = gidis_bilgisi.append(pd.DataFrame({
                                            'arac_index' : [vec],
                                            'index_from' : [dist_dict[vec][i]],
                                            'index_to' :[dist_dict[vec][i+1]],
                                            'mesafe_km' : [data['distance_matrix'][dist_dict[vec][i]][ dist_dict[vec][i+1]]],
                                            'zaman_dk' : [data['time_matrix'][dist_dict[vec][i]][ dist_dict[vec][i+1]]],
                                            'servis_suresi':[abs((13 + ((data['demands'][ dist_dict[vec][i+1]]-1) * 6 )))]
                                        }))
        
        
        points.append(dist_dict[vec][i])
        
        print(dist_dict[vec][i], '-->', dist_dict[vec][i+1], data['time_matrix'][dist_dict[vec][i]][ dist_dict[vec][i+1]])
        print('service time',abs((13 + ((data['demands'][ dist_dict[vec][i+1]]-1) * 6 ))))
#         print('service time',abs(data['demands'][ dist_dict[vec][i+1]] ))
        total_dur_min += data['time_matrix'][dist_dict[vec][i]][ dist_dict[vec][i+1]]
    total_sum_min += total_dur_min    
    sure[vec] = total_dur_min
    points.append(0)
    routes[vec] = points
    print('*******************************')
    print('Total Minutes of the trip:', total_sum_min)
    print('*******************************')

*******************************
Vehicle 0
----------
0 --> 143 127.88833333333334
service time 49
143 --> 117 110.38166666666666
service time 13
117 --> 0 17.30666666666667
service time 7
*******************************
Total Minutes of the trip: 255.57666666666665
*******************************
*******************************
Vehicle 1
----------
0 --> 0 0.0
service time 7
*******************************
Total Minutes of the trip: 0.0
*******************************
*******************************
Vehicle 2
----------
0 --> 0 0.0
service time 7
*******************************
Total Minutes of the trip: 0.0
*******************************
*******************************
Vehicle 3
----------
0 --> 0 0.0
service time 7
*******************************
Total Minutes of the trip: 0.0
*******************************
*******************************
Vehicle 4
----------
0 --> 0 0.0
service time 7
*******************************
Total Minutes of the trip: 0.0
*******************************
**

In [78]:
gidis_bilgisi = gidis_bilgisi[['arac_index','index_from','index_to','mesafe_km','zaman_dk', 'servis_suresi']]
gidis_bilgisi.head()

,arac_index,index_from,index_to,mesafe_km,zaman_dk,servis_suresi
0,0,0,143,181.8487,127.888333,49
0,0,143,117,160.6546,110.381667,13
0,0,117,0,17.3322,17.306667,7
0,1,0,0,0.0000,0.000000,7
0,2,0,0,0.0000,0.000000,7


In [79]:
gidis_bilgisi = gidis_bilgisi[~((gidis_bilgisi['index_to'] == 0) & (gidis_bilgisi['index_to'] == 0 ))]
gidis_bilgisi.head()

,arac_index,index_from,index_to,mesafe_km,zaman_dk,servis_suresi
0,0,0,143,181.8487,127.888333,49
0,0,143,117,160.6546,110.381667,13
0,18,0,88,21.6622,22.353333,13
0,18,88,69,6.3215,7.410000,43
0,18,69,136,2.4363,3.991667,25


In [80]:
gidis_bilgisi['magaza_top_sure'] = gidis_bilgisi['zaman_dk'] + gidis_bilgisi['servis_suresi']

In [81]:
gidis_bilgisi['kum_sure'] = gidis_bilgisi.groupby('arac_index')['magaza_top_sure'].transform(pd.Series.cumsum)
gidis_bilgisi

,arac_index,index_from,index_to,mesafe_km,zaman_dk,servis_suresi,magaza_top_sure,kum_sure
0,0,0,143,181.8487,127.888333,49,176.888333,176.888333
0,0,143,117,160.6546,110.381667,13,123.381667,300.27
0,18,0,88,21.6622,22.353333,13,35.353333,35.353333
0,18,88,69,6.3215,7.410000,43,50.41,85.763333
0,18,69,136,2.4363,3.991667,25,28.991667,114.755
0,326,0,111,19.7782,21.221667,37,58.221667,58.221667
0,326,111,131,3.1005,4.901667,49,53.901667,112.123333
0,326,131,126,0.8531,1.851667,43,44.851667,156.975
0,326,126,59,19.5543,19.346667,101,120.346667,277.321667
0,326,59,121,19.5183,20.565000,43,63.565,340.886667


In [82]:
gidis_bilgisi.groupby('arac_index')['kum_sure'].max().reset_index()

,arac_index,kum_sure
0,0,300.270000
1,18,114.755000
2,326,413.150000
3,327,383.233333
4,328,214.260000
5,329,283.343333
6,330,252.793333
7,331,345.015000
8,332,669.420000
9,333,688.245000


In [83]:
# gidis_bilgisi.loc[gidis_bilgisi['index_from'] < 61,'magaza_from'] = 'MERSIN DAGITIM MERKEZI'
# gidis_bilgisi

In [84]:
node_and_distances3 = node_and_distances2.rename(columns = {'index':'index_from', 'Birim':'birim_from','MAĞAZA ADI':'magaza_from'})

In [85]:
node_and_distances3.head(2)

,index_from,Node_From,Distance,Duration,birim_from,PALET_SAYISI,magaza_from,giden_arac_kapasite,ind,ind2
0,0,"36.892053081353,34.7391309093348","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99999.0,0,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0
1,1,"36.89205314135279,34.73913096933459","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99998.0,-18,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0


In [86]:
node_and_distances4 = node_and_distances2.rename(columns = {'index':'index_to', 'Birim':'birim_to','MAĞAZA ADI':'magaza_to','PALET_SAYISI':'palet_to','giden_arac_kapasite':'giden_arac_kapasite_to'})

In [87]:
 gidis_bilgisi.head()

,arac_index,index_from,index_to,mesafe_km,zaman_dk,servis_suresi,magaza_top_sure,kum_sure
0,0,0,143,181.8487,127.888333,49,176.888333,176.888333
0,0,143,117,160.6546,110.381667,13,123.381667,300.27
0,18,0,88,21.6622,22.353333,13,35.353333,35.353333
0,18,88,69,6.3215,7.410000,43,50.41,85.763333
0,18,69,136,2.4363,3.991667,25,28.991667,114.755


In [88]:
gidis_bilgisi2 = gidis_bilgisi.merge(node_and_distances3[['index_from','birim_from','magaza_from']], how = 'left',on = 'index_from')
gidis_bilgisi2.head()

,arac_index,index_from,index_to,mesafe_km,zaman_dk,servis_suresi,magaza_top_sure,kum_sure,birim_from,magaza_from
0,0,0,143,181.8487,127.888333,49,176.888333,176.888333,99999.0,MERSIN DAGITIM MERKEZI
1,0,143,117,160.6546,110.381667,13,123.381667,300.27,8167.0,ŞOK-NİGDE SELÇUK
2,18,0,88,21.6622,22.353333,13,35.353333,35.353333,99999.0,MERSIN DAGITIM MERKEZI
3,18,88,69,6.3215,7.410000,43,50.41,85.763333,5970.0,ŞOK-MERSİN KURDALİ
4,18,69,136,2.4363,3.991667,25,28.991667,114.755,6632.0,ŞOK-MERSİN BAHÇELİEVLER


In [89]:
gidis_bilgisi3 = gidis_bilgisi2.merge(node_and_distances4[['index_to','birim_to','magaza_to','palet_to','giden_arac_kapasite_to']], how = 'left',on = 'index_to')
gidis_bilgisi3.head()

,arac_index,index_from,index_to,mesafe_km,zaman_dk,servis_suresi,magaza_top_sure,kum_sure,birim_from,magaza_from,birim_to,magaza_to,palet_to,giden_arac_kapasite_to
0,0,0,143,181.8487,127.888333,49,176.888333,176.888333,99999.0,MERSIN DAGITIM MERKEZI,8167.0,ŞOK-NİGDE SELÇUK,7,18.0
1,0,143,117,160.6546,110.381667,13,123.381667,300.27,8167.0,ŞOK-NİGDE SELÇUK,8012.0,ŞOK-MERSİN TARSUS GAZİPAŞA,1,18.0
2,18,0,88,21.6622,22.353333,13,35.353333,35.353333,99999.0,MERSIN DAGITIM MERKEZI,5970.0,ŞOK-MERSİN KURDALİ,1,18.0
3,18,88,69,6.3215,7.410000,43,50.41,85.763333,5970.0,ŞOK-MERSİN KURDALİ,6632.0,ŞOK-MERSİN BAHÇELİEVLER,6,10.0
4,18,69,136,2.4363,3.991667,25,28.991667,114.755,6632.0,ŞOK-MERSİN BAHÇELİEVLER,1100.0,ŞOK-MERSİN ÇAMLIBEL,3,10.0


In [90]:
for i in range(len(node_and_distances)):
    print(i, node_and_distances.loc[dist_dict[0][0],'Distance'][i])
#     node_and_distances.loc[dist_dict[0][0],'Distance']

0 0.0
1 0.0
2 0.0
3 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0
9 0.0
10 0.0
11 0.0
12 0.0
13 0.0
14 0.0
15 0.0
16 0.0
17 0.0
18 0.0
19 0.0
20 0.0
21 0.0
22 0.0
23 0.0
24 0.0
25 0.0
26 0.0
27 0.0
28 0.0
29 0.0
30 0.0
31 0.0
32 0.0
33 0.0
34 0.0
35 0.0
36 0.0
37 0.0
38 0.0
39 0.0
40 0.0
41 0.0
42 0.0
43 0.0
44 0.0
45 0.0
46 0.0
47 0.0
48 0.0
49 0.0
50 0.0
51 0.0
52 0.0
53 0.0
54 0.0
55 0.0
56 0.0
57 0.0
58 0.0
59 0.0
60 0.0
61 28.3141
62 22.841900000000003
63 26.6754
64 31.08
65 25.3395
66 29.8857
67 23.8066
68 26.9898
69 25.2004
70 21.811400000000003
71 36.9199
72 31.2106
73 34.6546
74 22.5687
75 22.939400000000003
76 35.7747
77 25.0144
78 28.6684
79 26.7546
80 26.2837
81 34.8067
82 22.6431
83 163.3438
84 20.6801
85 32.3381
86 24.5288
87 23.7627
88 21.6622
89 22.3175
90 33.4537
91 26.5403
92 33.2675
93 32.3946
94 31.9988
95 177.6119
96 177.5605
97 177.83610000000002
98 177.3091
99 177.7473
100 32.9809
101 32.9211
102 21.3112
103 22.4592
104 30.2674
105 29.3508
106 23.2513
107 34.337300000000006


In [91]:
r = pd.DataFrame(columns=['index','routes'])
r['index']  = routes.keys()
r['routes']  = routes.values()
r.head()

,index,routes
0,0,"[0, 143, 117, 0]"
1,1,"[0, 0]"
2,2,"[0, 0]"
3,3,"[0, 0]"
4,4,"[0, 0]"


In [92]:
# node_and_distances = node_and_distances.drop(columns = ['index','level_0'])

In [93]:
node_and_distances.head()

,Node_From,Distance,Duration,Birim,PALET_SAYISI,MAĞAZA ADI,giden_arac_kapasite,ind,ind2
0,"36.892053081353,34.7391309093348","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99999.0,0,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0
1,"36.89205314135279,34.73913096933459","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99998.0,-18,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0
2,"36.89205314035279,34.73913096833459","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99997.0,-18,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0
3,"36.8920531393528,34.739130967334596","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99996.0,-18,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0
4,"36.8920531383528,34.7391309663346","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",99995.0,-18,MERSIN DAGITIM MERKEZI,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0


In [94]:
node_and_distances = node_and_distances.reset_index()
transform = dict(zip(node_and_distances.index, node_and_distances.Node_From))

In [95]:
transformed = []
for i in range(len(r)):
#     print(r.loc[i]['routes'])
    l =  list(map(transform.__getitem__, r.loc[i]['routes']))
    transformed.append(l)

In [96]:
full_routes = []
for i in range(len(transformed)):
    single_route = []
    if len(transformed[i])>2:
        for j in transformed[i]:
    #         print(tuple([float(j.split(',')[0]), float(j.split(',')[1])]))
            single_route.append(tuple([float(j.split(',')[0]), float(j.split(',')[1])]))
        full_routes.append(single_route)
full_routes

[[(36.892053081353, 34.7391309093348),
  (37.959764, 34.672599),
  (36.921755, 34.866891),
  (36.892053081353, 34.7391309093348)],
 [(36.892053081353, 34.7391309093348),
  (36.824983, 34.630049),
  (36.784898, 34.599241),
  (36.79, 34.6217),
  (36.892053081353, 34.7391309093348)],
 [(36.892053081353, 34.7391309093348),
  (36.824567, 34.647687),
  (36.807575, 34.635389),
  (36.8086, 34.628879),
  (36.89205308335299, 34.73913091133479),
  (36.918618, 34.886782),
  (36.917893, 34.887481),
  (36.932881, 34.885384),
  (36.892053081353, 34.7391309093348)],
 [(36.892053081353, 34.7391309093348),
  (36.927969, 34.929654),
  (37.966423, 34.64803),
  (37.965613, 34.653899),
  (37.976259, 34.672881),
  (36.931584, 34.861532),
  (36.892053081353, 34.7391309093348)],
 [(36.892053081353, 34.7391309093348),
  (36.920518, 34.883512),
  (36.939641, 34.885146),
  (36.945412, 34.87752),
  (36.944569, 34.873304),
  (36.938039, 34.869103),
  (36.922265, 34.881222),
  (36.909094, 34.892044),
  (36.920685, 3

In [97]:
list_colors = [
 #   "#00FF00",
 #   "#12FF00",
 #   "#24FF00",
 #   "#35FF00",
 #   "#47FF00",
 #   "#58FF00",
 #   "#6AFF00",
 #   "#7CFF00",
 #   "#8DFF00",
 #   "#9FFF00",
 #   "#B0FF00",
 #   "#C2FF00",
 #   "#D4FF00",
 #   "#E5FF00",
 #   "#F7FF00",
 #   "#FFF600",
 #   "#FFE400",
 #   "#FFD300",
 #   "#FFC100",
 #   "#FFAF00",
 #   "#FF9E00",
 #   "#FF8C00",
 #   "#FF7B00",
 #   "#FF6900",
 #   "#FF5700",
 #   "#FF4600",
 #   "#FF3400",
 #   "#FF2300",
 #   "#FF1100",
 #   "#FF0000",
     "#FE4EDA",
 "#FE6F5E",
 "#FE59C2",
 "#FD0E35",
 "#01796F",
 "#FD5E53",
 "#0FC0FC",
 "#FC0FC0",
 "#FC74FD",
 "#014421",
 "#03C03C",
 "#FDFD96",
 "#FCF75E",
 "#66023C",
 "#035096",
 "#013220",
 "#FC6C85",
 "#5218FA",
 "#FBEC5D",
 "#F9429E",
 "#08E8DE",
 "#059033",
 "#560319",
 "#0892D0",
 "#98FB98",
]
color_dict = {i: list_colors[i] for i in range(len(list_colors))}

In [98]:
len(list_colors)

25

In [99]:
len(full_routes)

16

In [100]:
m = folium.Map(location=[41, 29],zoom_start=5)
for i,j in zip(full_routes,list_colors):
#     print(i)
    folium.PolyLine(i,
                color=j,
                weight=2,
                opacity=0.8).add_to(m)
m

In [101]:
r2 = r.copy()
r2['capacities'] = capacities
r2.head()

,index,routes,capacities
0,0,"[0, 143, 117, 0]",10
1,1,"[0, 0]",10
2,2,"[0, 0]",10
3,3,"[0, 0]",10
4,4,"[0, 0]",10


In [102]:
r2['rota_mevcut_mu'] = r2['routes'].apply(lambda x: len(x)>2)
r2.head()

,index,routes,capacities,rota_mevcut_mu
0,0,"[0, 143, 117, 0]",10,True
1,1,"[0, 0]",10,False
2,2,"[0, 0]",10,False
3,3,"[0, 0]",10,False
4,4,"[0, 0]",10,False


In [103]:
r2 = r2[r2['rota_mevcut_mu'] == True]
r2 = r2.reset_index()
r2.head(2)

,level_0,index,routes,capacities,rota_mevcut_mu
0,0,0,"[0, 143, 117, 0]",10,True
1,18,18,"[0, 88, 69, 136, 0]",10,True


In [104]:
r2

,level_0,index,routes,capacities,rota_mevcut_mu
0,0,0,"[0, 143, 117, 0]",10,True
1,18,18,"[0, 88, 69, 136, 0]",10,True
2,326,326,"[0, 111, 131, 126, 59, 121, 120, 118, 0]",18,True
3,327,327,"[0, 119, 149, 150, 147, 160, 0]",18,True
4,328,328,"[0, 162, 114, 161, 122, 163, 158, 159, 156, 15...",18,True
5,329,329,"[0, 81, 97, 96, 95, 0]",18,True
6,330,330,"[0, 146, 155, 153, 0]",18,True
7,331,331,"[0, 84, 83, 99, 98, 0]",18,True
8,332,332,"[0, 94, 73, 90, 52, 85, 64, 141, 51, 113, 0]",18,True
9,333,333,"[0, 144, 148, 145, 116, 58, 92, 76, 71, 135, 0]",18,True


In [105]:
r2.capacities.value_counts()

18    14
10     2
Name: capacities, dtype: int64

In [106]:
rotalar = r2.copy()

In [107]:
mesafe = pd.DataFrame(data=dists[0],columns=['depoya_mesafe'])

In [108]:
mesafe = mesafe.reset_index()

In [109]:
mesafe_dict = dict(zip(mesafe.index, mesafe.depoya_mesafe))

In [110]:
def check(liste):
    try:
        return max(liste)
    except:
        return np.nan

In [111]:
def check2(liste):
    try:
        return liste
    except:
        return np.nan

In [112]:
mesafe_dict = dict(zip(mesafe.index, mesafe.depoya_mesafe))

In [113]:
magaza_dict = dict(zip(node_and_distances.index, node_and_distances['MAĞAZA ADI']))

In [114]:
birim_dict = dict(zip(node_and_distances.index, node_and_distances['Birim']))

In [115]:
def check(liste):
    try:
        return max(liste)
    except:
        return np.nan

In [116]:
def check2(liste):
    try:
        return liste
    except:
        return np.nan

In [117]:
def check_sum(liste):
    try:
        return sum(liste)
    except:
        return np.nan

In [118]:
palet_sayisi_dict = dict(zip(node_and_distances.index, node_and_distances.PALET_SAYISI))

In [119]:
D = []
    
itemlist=list(rotalar['routes'])  
    
import statistics
for A in itemlist:
    D.append(check2([magaza_dict.get(x, x) for x in A]))
rotalar['magaza_list'] = D

In [120]:
E = []
    
itemlist=list(rotalar['routes'])  
    
import statistics
for A in itemlist:
    E.append(check2([mesafe_dict.get(x, x) for x in A]))
rotalar['dist_list'] = E

In [121]:
F = []
    
itemlist=list(rotalar['routes'])  
    
import statistics
for A in itemlist:
    F.append(check2([palet_sayisi_dict.get(x, x) for x in A]))
rotalar['palet_list'] = F

In [122]:
G = []
    
itemlist=list(rotalar['routes'])  
    
import statistics
for A in itemlist:
    G.append(check_sum([palet_sayisi_dict.get(x, x) for x in A]))
rotalar['palet_toplam'] = G

In [123]:
I = []
    
itemlist=list(rotalar['routes'])  
    
import statistics
for A in itemlist:
    I.append(check2([birim_dict.get(x, x) for x in A]))
rotalar['birim_list'] = I

In [124]:
def check_total(liste):
    try:
        return len(liste)-2
    except:
        return np.nan

In [125]:
H = []
    
itemlist=list(rotalar['routes'])  
    
import statistics
for A in itemlist:
    H.append(check_total([palet_sayisi_dict.get(x, x) for x in A]))
rotalar['top_magaza_sayisi'] = H

In [126]:
D = []
    
itemlist=list(rotalar['routes'])  
    
import statistics
for A in itemlist:
    D.append(check2([mesafe_dict.get(x, x) for x in A]))
rotalar['dist_list'] = D

In [127]:
nigde = []
for i in range(len(rotalar)):
    liste = rotalar.loc[i,'magaza_list']
    for j in liste:
#         print(j)
        if 'NİĞDE' in j : 
            nigde.append(i)
            print(j,i)

ŞOK-NİĞDE FERTEK 3
ŞOK-NİĞDE KELEBEK 3
ŞOK-NİĞDE DERE 3
ŞOK-NİĞDE AYDINLIKEVLER 6
ŞOK-NİĞDE ŞAHİNALİ 6
ŞOK-NİĞDE YEŞİLBURÇ YOLU 6
ŞOK-NİĞDE 75. YIL 9
ŞOK-NİĞDE EFENDİBEY 9
ŞOK-NİĞDE ARAPDEDE 9
ŞOK-NİĞDE SELÇUK ÇARŞI 15
ŞOK-NİĞDE ŞAFAK 15
ŞOK-NİĞDE SAMATYA 15


In [128]:
len(palet_talep)

164

In [129]:
depot_nodes = {}
# depot_nodes[0] = str(0)
for i in range(0,61):
    depot_nodes[i] = 'DEPOT' + str(i)
for i in range(61, len(palet_talep)):
    depot_nodes[i] = str(i)
depot_nodes    

{0: 'DEPOT0',
 1: 'DEPOT1',
 2: 'DEPOT2',
 3: 'DEPOT3',
 4: 'DEPOT4',
 5: 'DEPOT5',
 6: 'DEPOT6',
 7: 'DEPOT7',
 8: 'DEPOT8',
 9: 'DEPOT9',
 10: 'DEPOT10',
 11: 'DEPOT11',
 12: 'DEPOT12',
 13: 'DEPOT13',
 14: 'DEPOT14',
 15: 'DEPOT15',
 16: 'DEPOT16',
 17: 'DEPOT17',
 18: 'DEPOT18',
 19: 'DEPOT19',
 20: 'DEPOT20',
 21: 'DEPOT21',
 22: 'DEPOT22',
 23: 'DEPOT23',
 24: 'DEPOT24',
 25: 'DEPOT25',
 26: 'DEPOT26',
 27: 'DEPOT27',
 28: 'DEPOT28',
 29: 'DEPOT29',
 30: 'DEPOT30',
 31: 'DEPOT31',
 32: 'DEPOT32',
 33: 'DEPOT33',
 34: 'DEPOT34',
 35: 'DEPOT35',
 36: 'DEPOT36',
 37: 'DEPOT37',
 38: 'DEPOT38',
 39: 'DEPOT39',
 40: 'DEPOT40',
 41: 'DEPOT41',
 42: 'DEPOT42',
 43: 'DEPOT43',
 44: 'DEPOT44',
 45: 'DEPOT45',
 46: 'DEPOT46',
 47: 'DEPOT47',
 48: 'DEPOT48',
 49: 'DEPOT49',
 50: 'DEPOT50',
 51: 'DEPOT51',
 52: 'DEPOT52',
 53: 'DEPOT53',
 54: 'DEPOT54',
 55: 'DEPOT55',
 56: 'DEPOT56',
 57: 'DEPOT57',
 58: 'DEPOT58',
 59: 'DEPOT59',
 60: 'DEPOT60',
 61: '61',
 62: '62',
 63: '63',
 64: '64',


In [130]:
# C = []
    
# itemlist=list(rotalar['routes'])  
    
# # import statistics
# # transform = dict(zip(data_merged2.MUSTERI_CLUSTER, data_merged2.STEP2))
# for A in itemlist:
#     C.append(([depot_nodes.get(x, x) for x in A]))
# #     A.discard(-1)
# #     C.append(np.fromiter([transform.get(x, x) for x in A], int).mean())
# rotalar['transformed_routes'] = C


In [131]:
itemlist

[[0, 143, 117, 0],
 [0, 88, 69, 136, 0],
 [0, 111, 131, 126, 59, 121, 120, 118, 0],
 [0, 119, 149, 150, 147, 160, 0],
 [0, 162, 114, 161, 122, 163, 158, 159, 156, 157, 0],
 [0, 81, 97, 96, 95, 0],
 [0, 146, 155, 153, 0],
 [0, 84, 83, 99, 98, 0],
 [0, 94, 73, 90, 52, 85, 64, 141, 51, 113, 0],
 [0, 144, 148, 145, 116, 58, 92, 76, 71, 135, 0],
 [0, 100, 101, 63, 54, 107, 128, 124, 53, 134, 61, 123, 0],
 [0, 66, 78, 0],
 [0, 93, 108, 79, 55, 103, 68, 139, 72, 0],
 [0, 62, 67, 80, 91, 56, 133, 105, 86, 65, 82, 0],
 [0,
  102,
  106,
  127,
  104,
  130,
  142,
  57,
  125,
  110,
  87,
  70,
  77,
  89,
  75,
  74,
  112,
  0],
 [0, 152, 154, 151, 115, 60, 132, 129, 137, 138, 109, 140, 0]]

In [132]:
C = []
genel = []
E = []
    
itemlist=list(rotalar['routes'])  

for A in itemlist:
    temp = (([depot_nodes.get(x, x) for x in A]))
    C.append(temp)
#     print(C)
    D = ''
    for i in temp:
#         print(i)
        D += i + ' '
#     print(D)
    E.append(D)

rotalar['transformed_routes'] = E


In [133]:
rotalar = rotalar.assign(seferler=rotalar.transformed_routes.str.split("DEPOT")).explode('seferler')
rotalar

,level_0,index,routes,capacities,rota_mevcut_mu,magaza_list,dist_list,palet_list,palet_toplam,birim_list,top_magaza_sayisi,transformed_routes,seferler
0,0,0,"[0, 143, 117, 0]",10,True,"[MERSIN DAGITIM MERKEZI, ŞOK-NİGDE SELÇUK, ŞOK...","[0.0, 181.8487, 18.8079, 0.0]","[0, 7, 1, 0]",8,"[99999.0, 8167.0, 8012.0, 99999.0]",2,DEPOT0 143 117 DEPOT0,
0,0,0,"[0, 143, 117, 0]",10,True,"[MERSIN DAGITIM MERKEZI, ŞOK-NİGDE SELÇUK, ŞOK...","[0.0, 181.8487, 18.8079, 0.0]","[0, 7, 1, 0]",8,"[99999.0, 8167.0, 8012.0, 99999.0]",2,DEPOT0 143 117 DEPOT0,0 143 117
0,0,0,"[0, 143, 117, 0]",10,True,"[MERSIN DAGITIM MERKEZI, ŞOK-NİGDE SELÇUK, ŞOK...","[0.0, 181.8487, 18.8079, 0.0]","[0, 7, 1, 0]",8,"[99999.0, 8167.0, 8012.0, 99999.0]",2,DEPOT0 143 117 DEPOT0,0
1,18,18,"[0, 88, 69, 136, 0]",10,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN KURDALİ, Ş...","[0.0, 21.6622, 25.2004, 22.4103, 0.0]","[0, 1, 6, 3, 0]",10,"[99999.0, 5970.0, 6632.0, 1100.0, 99999.0]",3,DEPOT0 88 69 136 DEPOT0,
1,18,18,"[0, 88, 69, 136, 0]",10,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN KURDALİ, Ş...","[0.0, 21.6622, 25.2004, 22.4103, 0.0]","[0, 1, 6, 3, 0]",10,"[99999.0, 5970.0, 6632.0, 1100.0, 99999.0]",3,DEPOT0 88 69 136 DEPOT0,0 88 69 136
1,18,18,"[0, 88, 69, 136, 0]",10,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN KURDALİ, Ş...","[0.0, 21.6622, 25.2004, 22.4103, 0.0]","[0, 1, 6, 3, 0]",10,"[99999.0, 5970.0, 6632.0, 1100.0, 99999.0]",3,DEPOT0 88 69 136 DEPOT0,0
2,326,326,"[0, 111, 131, 126, 59, 121, 120, 118, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN SİTELER, Ş...","[0.0, 19.7782, 20.5793, 20.9598, 0.0, 19.5183,...","[0, 5, 7, 6, -18, 6, 8, 1, 0]",15,"[99999.0, 9812.0, 2032.0, 8685.0, 99940.0, 113...",7,DEPOT0 111 131 126 DEPOT59 121 120 118 DEPOT0,
2,326,326,"[0, 111, 131, 126, 59, 121, 120, 118, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN SİTELER, Ş...","[0.0, 19.7782, 20.5793, 20.9598, 0.0, 19.5183,...","[0, 5, 7, 6, -18, 6, 8, 1, 0]",15,"[99999.0, 9812.0, 2032.0, 8685.0, 99940.0, 113...",7,DEPOT0 111 131 126 DEPOT59 121 120 118 DEPOT0,0 111 131 126
2,326,326,"[0, 111, 131, 126, 59, 121, 120, 118, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN SİTELER, Ş...","[0.0, 19.7782, 20.5793, 20.9598, 0.0, 19.5183,...","[0, 5, 7, 6, -18, 6, 8, 1, 0]",15,"[99999.0, 9812.0, 2032.0, 8685.0, 99940.0, 113...",7,DEPOT0 111 131 126 DEPOT59 121 120 118 DEPOT0,59 121 120 118
2,326,326,"[0, 111, 131, 126, 59, 121, 120, 118, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN SİTELER, Ş...","[0.0, 19.7782, 20.5793, 20.9598, 0.0, 19.5183,...","[0, 5, 7, 6, -18, 6, 8, 1, 0]",15,"[99999.0, 9812.0, 2032.0, 8685.0, 99940.0, 113...",7,DEPOT0 111 131 126 DEPOT59 121 120 118 DEPOT0,0


In [134]:
rotalar = rotalar[rotalar['seferler'].notnull()]
rotalar = rotalar[rotalar['seferler'] != '']
rotalar = rotalar[rotalar['seferler'] != '0 ']

In [135]:
rotalar

,level_0,index,routes,capacities,rota_mevcut_mu,magaza_list,dist_list,palet_list,palet_toplam,birim_list,top_magaza_sayisi,transformed_routes,seferler
0,0,0,"[0, 143, 117, 0]",10,True,"[MERSIN DAGITIM MERKEZI, ŞOK-NİGDE SELÇUK, ŞOK...","[0.0, 181.8487, 18.8079, 0.0]","[0, 7, 1, 0]",8,"[99999.0, 8167.0, 8012.0, 99999.0]",2,DEPOT0 143 117 DEPOT0,0 143 117
1,18,18,"[0, 88, 69, 136, 0]",10,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN KURDALİ, Ş...","[0.0, 21.6622, 25.2004, 22.4103, 0.0]","[0, 1, 6, 3, 0]",10,"[99999.0, 5970.0, 6632.0, 1100.0, 99999.0]",3,DEPOT0 88 69 136 DEPOT0,0 88 69 136
2,326,326,"[0, 111, 131, 126, 59, 121, 120, 118, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN SİTELER, Ş...","[0.0, 19.7782, 20.5793, 20.9598, 0.0, 19.5183,...","[0, 5, 7, 6, -18, 6, 8, 1, 0]",15,"[99999.0, 9812.0, 2032.0, 8685.0, 99940.0, 113...",7,DEPOT0 111 131 126 DEPOT59 121 120 118 DEPOT0,0 111 131 126
2,326,326,"[0, 111, 131, 126, 59, 121, 120, 118, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN SİTELER, Ş...","[0.0, 19.7782, 20.5793, 20.9598, 0.0, 19.5183,...","[0, 5, 7, 6, -18, 6, 8, 1, 0]",15,"[99999.0, 9812.0, 2032.0, 8685.0, 99940.0, 113...",7,DEPOT0 111 131 126 DEPOT59 121 120 118 DEPOT0,59 121 120 118
3,327,327,"[0, 119, 149, 150, 147, 160, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN TARSUS YAR...","[0.0, 23.2658, 182.9869, 182.4602, 180.5063, 2...","[0, 1, 5, 5, 5, 1, 0]",17,"[99999.0, 3290.0, 7790.0, 11660.0, 2839.0, 106...",5,DEPOT0 119 149 150 147 160 DEPOT0,0 119 149 150 147 160
4,328,328,"[0, 162, 114, 161, 122, 163, 158, 159, 156, 15...",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-TARSUS ŞEHİT İSHA...","[0.0, 19.5715, 21.8315, 22.6165, 22.8805, 21.9...","[0, 1, 1, 1, 1, 1, 1, 1, 6, 4, 0]",17,"[99999.0, 5347.0, 1605.0, 10812.0, 1240.0, 787...",9,DEPOT0 162 114 161 122 163 158 159 156 157 DEP...,0 162 114 161 122 163 158 159 156 157
5,329,329,"[0, 81, 97, 96, 95, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN GMK BULVAR...","[0.0, 34.8067, 177.83610000000002, 177.5605, 1...","[0, 1, 5, 6, 6, 0]",18,"[99999.0, 9683.0, 3363.0, 1232.0, 2239.0, 9999...",4,DEPOT0 81 97 96 95 DEPOT0,0 81 97 96 95
6,330,330,"[0, 146, 155, 153, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-NİĞDE AYDINLIKEVL...","[0.0, 178.1129, 179.7055, 181.36720000000005, ...","[0, 6, 6, 5, 0]",17,"[99999.0, 6046.0, 511.0, 11302.0, 99999.0]",3,DEPOT0 146 155 153 DEPOT0,0 146 155 153
7,331,331,"[0, 84, 83, 99, 98, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN GÜNDOĞDU, ...","[0.0, 20.6801, 163.3438, 177.7473, 177.3091, 0.0]","[0, 1, 7, 5, 5, 0]",18,"[99999.0, 839.0, 8384.0, 9086.0, 11401.0, 9999...",4,DEPOT0 84 83 99 98 DEPOT0,0 84 83 99 98
8,332,332,"[0, 94, 73, 90, 52, 85, 64, 141, 51, 113, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN MEZİTLİ YE...","[0.0, 31.9988, 34.6546, 33.4537, 0.0, 32.3381,...","[0, 6, 5, 7, -18, 7, 7, 4, -18, 6, 0]",6,"[99999.0, 8776.0, 5873.0, 7641.0, 99947.0, 774...",9,DEPOT0 94 73 90 DEPOT52 85 64 141 DEPOT51 113 ...,0 94 73 90


In [136]:
rotalar.seferler.nunique()

26

In [137]:
rotalar['seferler_depo'] = rotalar['seferler'].apply(lambda x: True if (len(x)==3) & (int(x[0])<=60) else False)

In [138]:
rotalar.seferler_depo.value_counts()

False    26
Name: seferler_depo, dtype: int64

In [139]:
rotalar = rotalar[rotalar.seferler_depo == False]

In [140]:
depolar = []
for i in range(0, 61):
    depolar.append(i)

In [141]:
#  depolar

In [142]:
itemlist=list(rotalar['seferler'])  
liste = []

for A in itemlist:
    tmp = A.split(' ')
    tmp = tmp[0:len(tmp)-1]
    intlist = [int(x) for x in tmp]
    intlist = [x for x in intlist if x not in depolar]
    liste.append(intlist)
rotalar['seferler2'] = liste 

In [143]:
rotalar.shape

(26, 15)

In [144]:
rotalar = rotalar[rotalar['seferler2'].map(lambda d: len(d)) > 0]
rotalar.shape

(26, 15)

In [145]:
rotalar.drop(columns = 'level_0', inplace = True)

In [146]:
rotalar = rotalar.reset_index()
rotalar

,level_0,index,routes,capacities,rota_mevcut_mu,magaza_list,dist_list,palet_list,palet_toplam,birim_list,top_magaza_sayisi,transformed_routes,seferler,seferler_depo,seferler2
0,0,0,"[0, 143, 117, 0]",10,True,"[MERSIN DAGITIM MERKEZI, ŞOK-NİGDE SELÇUK, ŞOK...","[0.0, 181.8487, 18.8079, 0.0]","[0, 7, 1, 0]",8,"[99999.0, 8167.0, 8012.0, 99999.0]",2,DEPOT0 143 117 DEPOT0,0 143 117,False,"[143, 117]"
1,1,18,"[0, 88, 69, 136, 0]",10,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN KURDALİ, Ş...","[0.0, 21.6622, 25.2004, 22.4103, 0.0]","[0, 1, 6, 3, 0]",10,"[99999.0, 5970.0, 6632.0, 1100.0, 99999.0]",3,DEPOT0 88 69 136 DEPOT0,0 88 69 136,False,"[88, 69, 136]"
2,2,326,"[0, 111, 131, 126, 59, 121, 120, 118, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN SİTELER, Ş...","[0.0, 19.7782, 20.5793, 20.9598, 0.0, 19.5183,...","[0, 5, 7, 6, -18, 6, 8, 1, 0]",15,"[99999.0, 9812.0, 2032.0, 8685.0, 99940.0, 113...",7,DEPOT0 111 131 126 DEPOT59 121 120 118 DEPOT0,0 111 131 126,False,"[111, 131, 126]"
3,2,326,"[0, 111, 131, 126, 59, 121, 120, 118, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN SİTELER, Ş...","[0.0, 19.7782, 20.5793, 20.9598, 0.0, 19.5183,...","[0, 5, 7, 6, -18, 6, 8, 1, 0]",15,"[99999.0, 9812.0, 2032.0, 8685.0, 99940.0, 113...",7,DEPOT0 111 131 126 DEPOT59 121 120 118 DEPOT0,59 121 120 118,False,"[121, 120, 118]"
4,3,327,"[0, 119, 149, 150, 147, 160, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN TARSUS YAR...","[0.0, 23.2658, 182.9869, 182.4602, 180.5063, 2...","[0, 1, 5, 5, 5, 1, 0]",17,"[99999.0, 3290.0, 7790.0, 11660.0, 2839.0, 106...",5,DEPOT0 119 149 150 147 160 DEPOT0,0 119 149 150 147 160,False,"[119, 149, 150, 147, 160]"
5,4,328,"[0, 162, 114, 161, 122, 163, 158, 159, 156, 15...",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-TARSUS ŞEHİT İSHA...","[0.0, 19.5715, 21.8315, 22.6165, 22.8805, 21.9...","[0, 1, 1, 1, 1, 1, 1, 1, 6, 4, 0]",17,"[99999.0, 5347.0, 1605.0, 10812.0, 1240.0, 787...",9,DEPOT0 162 114 161 122 163 158 159 156 157 DEP...,0 162 114 161 122 163 158 159 156 157,False,"[162, 114, 161, 122, 163, 158, 159, 156, 157]"
6,5,329,"[0, 81, 97, 96, 95, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN GMK BULVAR...","[0.0, 34.8067, 177.83610000000002, 177.5605, 1...","[0, 1, 5, 6, 6, 0]",18,"[99999.0, 9683.0, 3363.0, 1232.0, 2239.0, 9999...",4,DEPOT0 81 97 96 95 DEPOT0,0 81 97 96 95,False,"[81, 97, 96, 95]"
7,6,330,"[0, 146, 155, 153, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-NİĞDE AYDINLIKEVL...","[0.0, 178.1129, 179.7055, 181.36720000000005, ...","[0, 6, 6, 5, 0]",17,"[99999.0, 6046.0, 511.0, 11302.0, 99999.0]",3,DEPOT0 146 155 153 DEPOT0,0 146 155 153,False,"[146, 155, 153]"
8,7,331,"[0, 84, 83, 99, 98, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN GÜNDOĞDU, ...","[0.0, 20.6801, 163.3438, 177.7473, 177.3091, 0.0]","[0, 1, 7, 5, 5, 0]",18,"[99999.0, 839.0, 8384.0, 9086.0, 11401.0, 9999...",4,DEPOT0 84 83 99 98 DEPOT0,0 84 83 99 98,False,"[84, 83, 99, 98]"
9,8,332,"[0, 94, 73, 90, 52, 85, 64, 141, 51, 113, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN MEZİTLİ YE...","[0.0, 31.9988, 34.6546, 33.4537, 0.0, 32.3381,...","[0, 6, 5, 7, -18, 7, 7, 4, -18, 6, 0]",6,"[99999.0, 8776.0, 5873.0, 7641.0, 99947.0, 774...",9,DEPOT0 94 73 90 DEPOT52 85 64 141 DEPOT51 113 ...,0 94 73 90,False,"[94, 73, 90]"


In [147]:
rotalar.shape

(26, 15)

In [148]:
rotalar[['capacities','level_0']].value_counts()

capacities  level_0
18          8          3
            10         3
            2          2
            9          2
            12         2
            13         2
            14         2
            15         2
10          0          1
            1          1
18          3          1
            4          1
            5          1
            6          1
            7          1
            11         1
dtype: int64

In [149]:
rotalar.head()

,level_0,index,routes,capacities,rota_mevcut_mu,magaza_list,dist_list,palet_list,palet_toplam,birim_list,top_magaza_sayisi,transformed_routes,seferler,seferler_depo,seferler2
0,0,0,"[0, 143, 117, 0]",10,True,"[MERSIN DAGITIM MERKEZI, ŞOK-NİGDE SELÇUK, ŞOK...","[0.0, 181.8487, 18.8079, 0.0]","[0, 7, 1, 0]",8,"[99999.0, 8167.0, 8012.0, 99999.0]",2,DEPOT0 143 117 DEPOT0,0 143 117,False,"[143, 117]"
1,1,18,"[0, 88, 69, 136, 0]",10,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN KURDALİ, Ş...","[0.0, 21.6622, 25.2004, 22.4103, 0.0]","[0, 1, 6, 3, 0]",10,"[99999.0, 5970.0, 6632.0, 1100.0, 99999.0]",3,DEPOT0 88 69 136 DEPOT0,0 88 69 136,False,"[88, 69, 136]"
2,2,326,"[0, 111, 131, 126, 59, 121, 120, 118, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN SİTELER, Ş...","[0.0, 19.7782, 20.5793, 20.9598, 0.0, 19.5183,...","[0, 5, 7, 6, -18, 6, 8, 1, 0]",15,"[99999.0, 9812.0, 2032.0, 8685.0, 99940.0, 113...",7,DEPOT0 111 131 126 DEPOT59 121 120 118 DEPOT0,0 111 131 126,False,"[111, 131, 126]"
3,2,326,"[0, 111, 131, 126, 59, 121, 120, 118, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN SİTELER, Ş...","[0.0, 19.7782, 20.5793, 20.9598, 0.0, 19.5183,...","[0, 5, 7, 6, -18, 6, 8, 1, 0]",15,"[99999.0, 9812.0, 2032.0, 8685.0, 99940.0, 113...",7,DEPOT0 111 131 126 DEPOT59 121 120 118 DEPOT0,59 121 120 118,False,"[121, 120, 118]"
4,3,327,"[0, 119, 149, 150, 147, 160, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN TARSUS YAR...","[0.0, 23.2658, 182.9869, 182.4602, 180.5063, 2...","[0, 1, 5, 5, 5, 1, 0]",17,"[99999.0, 3290.0, 7790.0, 11660.0, 2839.0, 106...",5,DEPOT0 119 149 150 147 160 DEPOT0,0 119 149 150 147 160,False,"[119, 149, 150, 147, 160]"


### OUTPUT FORMAT

In [150]:
rotalar

,level_0,index,routes,capacities,rota_mevcut_mu,magaza_list,dist_list,palet_list,palet_toplam,birim_list,top_magaza_sayisi,transformed_routes,seferler,seferler_depo,seferler2
0,0,0,"[0, 143, 117, 0]",10,True,"[MERSIN DAGITIM MERKEZI, ŞOK-NİGDE SELÇUK, ŞOK...","[0.0, 181.8487, 18.8079, 0.0]","[0, 7, 1, 0]",8,"[99999.0, 8167.0, 8012.0, 99999.0]",2,DEPOT0 143 117 DEPOT0,0 143 117,False,"[143, 117]"
1,1,18,"[0, 88, 69, 136, 0]",10,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN KURDALİ, Ş...","[0.0, 21.6622, 25.2004, 22.4103, 0.0]","[0, 1, 6, 3, 0]",10,"[99999.0, 5970.0, 6632.0, 1100.0, 99999.0]",3,DEPOT0 88 69 136 DEPOT0,0 88 69 136,False,"[88, 69, 136]"
2,2,326,"[0, 111, 131, 126, 59, 121, 120, 118, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN SİTELER, Ş...","[0.0, 19.7782, 20.5793, 20.9598, 0.0, 19.5183,...","[0, 5, 7, 6, -18, 6, 8, 1, 0]",15,"[99999.0, 9812.0, 2032.0, 8685.0, 99940.0, 113...",7,DEPOT0 111 131 126 DEPOT59 121 120 118 DEPOT0,0 111 131 126,False,"[111, 131, 126]"
3,2,326,"[0, 111, 131, 126, 59, 121, 120, 118, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN SİTELER, Ş...","[0.0, 19.7782, 20.5793, 20.9598, 0.0, 19.5183,...","[0, 5, 7, 6, -18, 6, 8, 1, 0]",15,"[99999.0, 9812.0, 2032.0, 8685.0, 99940.0, 113...",7,DEPOT0 111 131 126 DEPOT59 121 120 118 DEPOT0,59 121 120 118,False,"[121, 120, 118]"
4,3,327,"[0, 119, 149, 150, 147, 160, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN TARSUS YAR...","[0.0, 23.2658, 182.9869, 182.4602, 180.5063, 2...","[0, 1, 5, 5, 5, 1, 0]",17,"[99999.0, 3290.0, 7790.0, 11660.0, 2839.0, 106...",5,DEPOT0 119 149 150 147 160 DEPOT0,0 119 149 150 147 160,False,"[119, 149, 150, 147, 160]"
5,4,328,"[0, 162, 114, 161, 122, 163, 158, 159, 156, 15...",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-TARSUS ŞEHİT İSHA...","[0.0, 19.5715, 21.8315, 22.6165, 22.8805, 21.9...","[0, 1, 1, 1, 1, 1, 1, 1, 6, 4, 0]",17,"[99999.0, 5347.0, 1605.0, 10812.0, 1240.0, 787...",9,DEPOT0 162 114 161 122 163 158 159 156 157 DEP...,0 162 114 161 122 163 158 159 156 157,False,"[162, 114, 161, 122, 163, 158, 159, 156, 157]"
6,5,329,"[0, 81, 97, 96, 95, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN GMK BULVAR...","[0.0, 34.8067, 177.83610000000002, 177.5605, 1...","[0, 1, 5, 6, 6, 0]",18,"[99999.0, 9683.0, 3363.0, 1232.0, 2239.0, 9999...",4,DEPOT0 81 97 96 95 DEPOT0,0 81 97 96 95,False,"[81, 97, 96, 95]"
7,6,330,"[0, 146, 155, 153, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-NİĞDE AYDINLIKEVL...","[0.0, 178.1129, 179.7055, 181.36720000000005, ...","[0, 6, 6, 5, 0]",17,"[99999.0, 6046.0, 511.0, 11302.0, 99999.0]",3,DEPOT0 146 155 153 DEPOT0,0 146 155 153,False,"[146, 155, 153]"
8,7,331,"[0, 84, 83, 99, 98, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN GÜNDOĞDU, ...","[0.0, 20.6801, 163.3438, 177.7473, 177.3091, 0.0]","[0, 1, 7, 5, 5, 0]",18,"[99999.0, 839.0, 8384.0, 9086.0, 11401.0, 9999...",4,DEPOT0 84 83 99 98 DEPOT0,0 84 83 99 98,False,"[84, 83, 99, 98]"
9,8,332,"[0, 94, 73, 90, 52, 85, 64, 141, 51, 113, 0]",18,True,"[MERSIN DAGITIM MERKEZI, ŞOK-MERSİN MEZİTLİ YE...","[0.0, 31.9988, 34.6546, 33.4537, 0.0, 32.3381,...","[0, 6, 5, 7, -18, 7, 7, 4, -18, 6, 0]",6,"[99999.0, 8776.0, 5873.0, 7641.0, 99947.0, 774...",9,DEPOT0 94 73 90 DEPOT52 85 64 141 DEPOT51 113 ...,0 94 73 90,False,"[94, 73, 90]"


In [151]:
output = rotalar.copy()

In [152]:
output.level_0.nunique()

16

In [153]:
output.level_0.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [154]:
output.seferler.nunique()

26

In [155]:
output['index'].nunique()

16

In [156]:
# # output.rename(columns = {'index':'Araç_ID'}, inplace = True)
# output['Araç_ID'] = 

In [157]:
# output['Araç_ID'] = output.groupby(['Araç_ID']).ngroup()

In [158]:
# output['Sefer_Numarası'] = output.groupby(['Araç_ID']).cumcount()+1

In [159]:
#output2 = output.drop(columns = 'level_0')
#output2 = output2.reset_index()
#output2

In [160]:
output_unique = output.drop_duplicates(subset='level_0',keep='first')

In [161]:
output3 = output_unique.explode(['routes', 'birim_list','magaza_list', 'palet_list', 'dist_list'])
output3['dist_list'] = output3['dist_list'] * 2

In [162]:
output3['level_0'].nunique()

16

In [163]:
output3['seferler'].nunique()

16

In [164]:
output3['magaza_list'].nunique()

104

In [165]:
df = output3.copy()

In [166]:
df.reset_index(inplace=True, drop=True)
df.head(5)

,level_0,index,routes,capacities,rota_mevcut_mu,magaza_list,dist_list,palet_list,palet_toplam,birim_list,top_magaza_sayisi,transformed_routes,seferler,seferler_depo,seferler2
0,0,0,0,10,True,MERSIN DAGITIM MERKEZI,0.0,0,8,99999.0,2,DEPOT0 143 117 DEPOT0,0 143 117,False,"[143, 117]"
1,0,0,143,10,True,ŞOK-NİGDE SELÇUK,363.6974,7,8,8167.0,2,DEPOT0 143 117 DEPOT0,0 143 117,False,"[143, 117]"
2,0,0,117,10,True,ŞOK-MERSİN TARSUS GAZİPAŞA,37.6158,1,8,8012.0,2,DEPOT0 143 117 DEPOT0,0 143 117,False,"[143, 117]"
3,0,0,0,10,True,MERSIN DAGITIM MERKEZI,0.0,0,8,99999.0,2,DEPOT0 143 117 DEPOT0,0 143 117,False,"[143, 117]"
4,1,18,0,10,True,MERSIN DAGITIM MERKEZI,0.0,0,10,99999.0,3,DEPOT0 88 69 136 DEPOT0,0 88 69 136,False,"[88, 69, 136]"


In [167]:
df.routes = df.routes.astype(float)

In [168]:
df.loc[df['routes'] <=60, 'routes'] = 0

In [169]:
df = df.groupby("level_0").apply(lambda x: x.iloc[:-1])

In [170]:
df.head()

level_0  index  routes  capacities  rota_mevcut_mu  \
level_0                                                         
0       0        0      0     0.0          10            True   
        1        0      0   143.0          10            True   
        2        0      0   117.0          10            True   
1       4        1     18     0.0          10            True   
        5        1     18    88.0          10            True   

                          magaza_list dist_list palet_list  palet_toplam  \
level_0                                                                    
0       0      MERSIN DAGITIM MERKEZI       0.0          0             8   
        1            ŞOK-NİGDE SELÇUK  363.6974          7             8   
        2  ŞOK-MERSİN TARSUS GAZİPAŞA   37.6158          1             8   
1       4      MERSIN DAGITIM MERKEZI       0.0          0            10   
        5          ŞOK-MERSİN KURDALİ   43.3244          1            10   

          birim_list  top_magaza_sayisi        transformed_routes  \
level_0                                                             
0       0    99999.0                  2    DEPOT0 143 117 DEPOT0    
        1     8167.0                  2    DEPOT0 143 117 DEPOT0    
        2     8012.0                  2    DEPOT0 143 117 DEPOT0    
1       4    99999.0                  3  DEPOT0 88 69 136 DEPOT0    
        5     5970.0                  3  DEPOT0 88 69 136 DEPOT0    

               seferler  seferler_depo      seferler2  
level_0                                                
0       0    0 143 117           False     [143, 117]  
        1    0 143 117           False     [143, 117]  
        2    0 143 117           False     [143, 117]  
1       4  0 88 69 136           False  [88, 69, 136]  
        5  0 88 69 136           False  [88, 69, 136]

In [171]:
df.reset_index(drop=True,inplace=True)

In [172]:
%%time
count = 0
all_df = []
for sefer_ in df.level_0.unique():
    arac = df[df.level_0 == sefer_]
    arac.reset_index(drop=True, inplace=True)
    for idx in arac.index:
        if arac.loc[idx, 'routes'] <= 60:
            count += 1
        else:
            pass
        arac.loc[idx, 'sefer_id'] = count
    
    all_df.append(arac)

df_ = pd.concat(all_df)
    

CPU times: user 32.2 ms, sys: 1.71 ms, total: 33.9 ms
Wall time: 32.7 ms


In [173]:
df_[['level_0','routes','sefer_id']]

,level_0,routes,sefer_id
0,0,0.0,1.0
1,0,143.0,1.0
2,0,117.0,1.0
0,1,0.0,2.0
1,1,88.0,2.0
2,1,69.0,2.0
3,1,136.0,2.0
0,2,0.0,3.0
1,2,111.0,3.0
2,2,131.0,3.0


In [174]:
df_[df_['palet_list']!= -18]['palet_list'].sum()

413

In [175]:
output3 = df_.copy()

In [176]:
output3.rename(columns={'level_0':'arac_id'}, inplace = True)

In [177]:
output3 = output3.merge(output3.groupby('sefer_id').size().reset_index(name='count'), on = 'sefer_id', how = 'left')
output3 = output3[output3['count']>1]


In [178]:
# df.groupby('seferler').size().reset_index(name='count')

In [179]:
output4 = output3[['arac_id','capacities','routes', 'birim_list','magaza_list', 'palet_list', 'dist_list', 'sefer_id']]
output4.loc[output4['palet_list'] == -18, 'palet_list'] = 0
output4.reset_index(drop = True, inplace = True)
output4

,arac_id,capacities,routes,birim_list,magaza_list,palet_list,dist_list,sefer_id
0,0,10,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,1.0
1,0,10,143.0,8167.0,ŞOK-NİGDE SELÇUK,7,363.6974,1.0
2,0,10,117.0,8012.0,ŞOK-MERSİN TARSUS GAZİPAŞA,1,37.6158,1.0
3,1,10,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,2.0
4,1,10,88.0,5970.0,ŞOK-MERSİN KURDALİ,1,43.3244,2.0
5,1,10,69.0,6632.0,ŞOK-MERSİN BAHÇELİEVLER,6,50.4008,2.0
6,1,10,136.0,1100.0,ŞOK-MERSİN ÇAMLIBEL,3,44.8206,2.0
7,2,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,3.0
8,2,18,111.0,9812.0,ŞOK-MERSİN SİTELER,5,39.5564,3.0
9,2,18,131.0,2032.0,ŞOK-MERSİN YENİMAHALLE,7,41.1586,3.0


In [180]:
output4.arac_id.nunique()

16

In [181]:
output4.sefer_id.nunique()

26

In [182]:
output4.palet_list.sum()

413

In [183]:
output4_unq = output4.drop_duplicates('sefer_id', keep = 'first')

In [184]:
palet_toplam_mdl = output4.groupby('sefer_id')['palet_list'].sum().reset_index()

In [185]:
palet_toplam_mdl = palet_toplam_mdl.merge(output4_unq[['sefer_id','capacities']], on ='sefer_id', how = 'left')

In [186]:
palet_toplam_mdl = palet_toplam_mdl[palet_toplam_mdl['palet_list'] != 0]

In [187]:
palet_toplam_mdl['Doluluk'] = palet_toplam_mdl['palet_list']/palet_toplam_mdl['capacities']

In [188]:
palet_toplam_mdl

,sefer_id,palet_list,capacities,Doluluk
0,1.0,8,10,0.8
1,2.0,10,10,1.0
2,3.0,18,18,1.0
3,4.0,15,18,0.833333
4,5.0,17,18,0.944444
5,6.0,17,18,0.944444
6,7.0,18,18,1.0
7,8.0,17,18,0.944444
8,9.0,18,18,1.0
9,10.0,18,18,1.0


In [189]:
palet_toplam_mdl['Doluluk'].mean()

0.9132478632478632

In [190]:
max_dist = output4.groupby('sefer_id')['dist_list'].max().reset_index().rename(columns = {'dist_list':'max_distance'})
max_dist.head()

,sefer_id,max_distance
0,1.0,363.6974
1,2.0,50.4008
2,3.0,41.9196
3,4.0,41.4390
4,5.0,365.9738


In [191]:
output4 = output4.merge(max_dist, on = 'sefer_id', how = 'left')

In [192]:
output4.head(100)

,arac_id,capacities,routes,birim_list,magaza_list,palet_list,dist_list,sefer_id,max_distance
0,0,10,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,1.0,363.6974
1,0,10,143.0,8167.0,ŞOK-NİGDE SELÇUK,7,363.6974,1.0,363.6974
2,0,10,117.0,8012.0,ŞOK-MERSİN TARSUS GAZİPAŞA,1,37.6158,1.0,363.6974
3,1,10,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,2.0,50.4008
4,1,10,88.0,5970.0,ŞOK-MERSİN KURDALİ,1,43.3244,2.0,50.4008
5,1,10,69.0,6632.0,ŞOK-MERSİN BAHÇELİEVLER,6,50.4008,2.0,50.4008
6,1,10,136.0,1100.0,ŞOK-MERSİN ÇAMLIBEL,3,44.8206,2.0,50.4008
7,2,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,3.0,41.9196
8,2,18,111.0,9812.0,ŞOK-MERSİN SİTELER,5,39.5564,3.0,41.9196
9,2,18,131.0,2032.0,ŞOK-MERSİN YENİMAHALLE,7,41.1586,3.0,41.9196


In [193]:
output4.merge(output4.groupby('sefer_id').size().reset_index(name='count'), on = 'sefer_id', how = 'left')
# df.groupby(['id', 'date1']).size().reset_index(name='count')

,arac_id,capacities,routes,birim_list,magaza_list,palet_list,dist_list,sefer_id,max_distance,count
0,0,10,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,1.0,363.6974,3
1,0,10,143.0,8167.0,ŞOK-NİGDE SELÇUK,7,363.6974,1.0,363.6974,3
2,0,10,117.0,8012.0,ŞOK-MERSİN TARSUS GAZİPAŞA,1,37.6158,1.0,363.6974,3
3,1,10,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,2.0,50.4008,4
4,1,10,88.0,5970.0,ŞOK-MERSİN KURDALİ,1,43.3244,2.0,50.4008,4
5,1,10,69.0,6632.0,ŞOK-MERSİN BAHÇELİEVLER,6,50.4008,2.0,50.4008,4
6,1,10,136.0,1100.0,ŞOK-MERSİN ÇAMLIBEL,3,44.8206,2.0,50.4008,4
7,2,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,3.0,41.9196,4
8,2,18,111.0,9812.0,ŞOK-MERSİN SİTELER,5,39.5564,3.0,41.9196,4
9,2,18,131.0,2032.0,ŞOK-MERSİN YENİMAHALLE,7,41.1586,3.0,41.9196,4


In [194]:
gidis_bilgisi3.head(2)

,arac_index,index_from,index_to,mesafe_km,zaman_dk,servis_suresi,magaza_top_sure,kum_sure,birim_from,magaza_from,birim_to,magaza_to,palet_to,giden_arac_kapasite_to
0,0,0,143,181.8487,127.888333,49,176.888333,176.888333,99999.0,MERSIN DAGITIM MERKEZI,8167.0,ŞOK-NİGDE SELÇUK,7,18.0
1,0,143,117,160.6546,110.381667,13,123.381667,300.27,8167.0,ŞOK-NİGDE SELÇUK,8012.0,ŞOK-MERSİN TARSUS GAZİPAŞA,1,18.0


In [195]:
output4_sure = output4.merge(gidis_bilgisi3[['birim_to','mesafe_km','zaman_dk','servis_suresi','magaza_top_sure','kum_sure']], how = 'left', left_on = 'birim_list', right_on = 'birim_to')

In [196]:
output4_sure.drop(columns = 'birim_to', inplace = True)
output4_sure.rename(columns = {'dist_list':'depo_magaza_mesafe','max_distance':'sefer_max_km', 'mesafe_km':'from_mesafe_to', 'zaman_dk':'from_sure_to', 'servis_suresi':'magaza_servis_suresi'}, inplace = True)
output4_sure

,arac_id,capacities,routes,birim_list,magaza_list,palet_list,depo_magaza_mesafe,sefer_id,sefer_max_km,from_mesafe_to,from_sure_to,magaza_servis_suresi,magaza_top_sure,kum_sure
0,0,10,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,1.0,363.6974,NaN,NaN,NaN,NaN,NaN
1,0,10,143.0,8167.0,ŞOK-NİGDE SELÇUK,7,363.6974,1.0,363.6974,181.8487,127.888333,49,176.888333,176.888333
2,0,10,117.0,8012.0,ŞOK-MERSİN TARSUS GAZİPAŞA,1,37.6158,1.0,363.6974,160.6546,110.381667,13,123.381667,300.27
3,1,10,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,2.0,50.4008,NaN,NaN,NaN,NaN,NaN
4,1,10,88.0,5970.0,ŞOK-MERSİN KURDALİ,1,43.3244,2.0,50.4008,21.6622,22.353333,13,35.353333,35.353333
5,1,10,69.0,6632.0,ŞOK-MERSİN BAHÇELİEVLER,6,50.4008,2.0,50.4008,6.3215,7.410000,43,50.41,85.763333
6,1,10,136.0,1100.0,ŞOK-MERSİN ÇAMLIBEL,3,44.8206,2.0,50.4008,2.4363,3.991667,25,28.991667,114.755
7,2,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,3.0,41.9196,NaN,NaN,NaN,NaN,NaN
8,2,18,111.0,9812.0,ŞOK-MERSİN SİTELER,5,39.5564,3.0,41.9196,19.7782,21.221667,37,58.221667,58.221667
9,2,18,131.0,2032.0,ŞOK-MERSİN YENİMAHALLE,7,41.1586,3.0,41.9196,3.1005,4.901667,49,53.901667,112.123333


In [197]:
output4.to_csv('veriler/output_rotalar_' + str(siparis_gunu) +'_aralik.csv', index = False)

In [198]:
output4_sure.to_csv('veriler/output_rotalar_sureli' + str(siparis_gunu) +'_aralik.csv', index = False)

### MALIYET

### Gerçek Maliyet 

In [199]:
sefer_ozet = pd.read_excel('veriler/sok_palet_magaza.xlsx',sheet_name='Sefer Özet')
sefer_ozet.head(1)

,Cari No,Sefer Numarası,Plaka,Araç Tipi,Rulot Sayısı,Palet Sayısı,Toplam Rulot ve Palet Sayısı,Malzeme Palet Sayısı,Toplanan Koli,Mağaza,Koli Sayısı,Değerli Kasa,Maliyet Tutar,Çıkış Tarihi,Dönüş Tarihi,Sevkiyat Süre Saat,Sefer Durumu,Mağaza Sayısı,Doluluk Oranı(%),Toplanan Hacim,Toplanan Ağırlık (kg),Rulot Kapasitesi,En Uzak Mağaza Km,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
0,251062,7169955,06DU1114,18 PLT,0,1,1,0,92,MERSİN TARSUS ALTAYLILAR,51.81,0,27114.2179,2021-12-01 01:14:19,2021-12-01 12:34:55,11 sa 21 dk,TAMAMLANDI,6,113,7579.0,41,23,65.0,NaN,Satır Etiketleri,Araç Tipi,Ortalama En Uzak Mağaza Km,%Janu21%


In [200]:
sefer_ozet['Çıkış Tarihi'] = pd.to_datetime(sefer_ozet['Çıkış Tarihi'], format = '%Y-%m-%d').copy()

In [201]:
sefer_ozet = sefer_ozet[(sefer_ozet['Çıkış Tarihi'].dt.year == 2021) & (sefer_ozet['Çıkış Tarihi'].dt.month == 12) &\
(sefer_ozet['Çıkış Tarihi'].dt.day == siparis_gunu)]

In [202]:
sefer_ozet = sefer_ozet[~sefer_ozet.Mağaza.str.contains('BTT')]

In [203]:
sefer_ozet = sefer_ozet[sefer_ozet['Palet Sayısı'] != 0]

In [204]:
sefer_ozet['Palet Sayısı'].sum()

413

In [205]:
sefer_ozet.to_excel('17_aralık_sefer_özet.xlsx', index = False)

In [206]:
sefer_ozet.head()

,Cari No,Sefer Numarası,Plaka,Araç Tipi,Rulot Sayısı,Palet Sayısı,Toplam Rulot ve Palet Sayısı,Malzeme Palet Sayısı,Toplanan Koli,Mağaza,Koli Sayısı,Değerli Kasa,Maliyet Tutar,Çıkış Tarihi,Dönüş Tarihi,Sevkiyat Süre Saat,Sefer Durumu,Mağaza Sayısı,Doluluk Oranı(%),Toplanan Hacim,Toplanan Ağırlık (kg),Rulot Kapasitesi,En Uzak Mağaza Km,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
2525,251062,7258421,33CFN11,18 PLT,0,7,8,0,558,MERSİN 34.CADDE,412.78,1,54454.1623,2021-12-17 00:58:10,2021-12-17 12:46:15,11 sa 48 dk,TAMAMLANDI,3,57,99361.0,317,30,80.0,NaN,NaN,NaN,NaN,NaN
2526,251062,7258421,33CFN11,18 PLT,0,7,8,0,694,MERSİN KAMPÜS,534.11,1,41757.1641,2021-12-17 00:58:10,2021-12-17 12:46:15,11 sa 48 dk,TAMAMLANDI,3,57,105192.0,356,30,80.0,NaN,NaN,NaN,NaN,NaN
2527,251062,7258421,33CFN11,18 PLT,0,1,1,0,109,MERSİN PARİS,40.81,0,40884.8421,2021-12-17 00:58:10,2021-12-17 12:46:15,11 sa 48 dk,TAMAMLANDI,3,57,3200.0,22,30,80.0,NaN,NaN,NaN,NaN,NaN
2528,251062,7258425,34UG2185,18 PLT,0,5,7,0,434,MERSİN TARSUS 100.YIL,306.07,1,26905.8693,2021-12-17 01:33:56,2021-12-17 11:59:54,10 sa 26 dk,TAMAMLANDI,6,96,60195.0,251,23,65.0,NaN,NaN,NaN,NaN,NaN
2529,251062,7258425,34UG2185,18 PLT,0,1,1,0,185,MERSİN TARSUS AKŞEMSETTİN,67.04,0,30939.4226,2021-12-17 01:33:56,2021-12-17 11:59:54,10 sa 26 dk,TAMAMLANDI,6,96,862.0,40,23,65.0,NaN,NaN,NaN,NaN,NaN


In [207]:
toplam_palet = sefer_ozet.groupby('Sefer Numarası')['Palet Sayısı'].sum().reset_index()
toplam_palet.head(2)

,Sefer Numarası,Palet Sayısı
0,7258421,15
1,7258425,18


In [208]:
toplam_palet.rename(columns = {'Palet Sayısı':'Sefer_Toplam_Palet'}, inplace = True)
toplam_palet.head(2)


,Sefer Numarası,Sefer_Toplam_Palet
0,7258421,15
1,7258425,18


In [209]:
toplam_palet = toplam_palet[toplam_palet['Sefer_Toplam_Palet'] != 0]

In [210]:
toplam_palet.shape

(27, 2)

In [211]:
sefer_ozet_unq = sefer_ozet.drop_duplicates('Sefer Numarası', keep = 'first')

In [212]:
toplam_palet = toplam_palet.merge(sefer_ozet_unq[['Sefer Numarası', 'Araç Tipi']], on = 'Sefer Numarası', how = 'left')
toplam_palet

,Sefer Numarası,Sefer_Toplam_Palet,Araç Tipi
0,7258421,15,18 PLT
1,7258425,18,18 PLT
2,7260533,17,18 PLT
3,7260581,18,18 PLT
4,7260626,15,18 PLT
5,7260658,15,18 PLT
6,7260692,15,18 PLT
7,7260742,15,18 PLT
8,7260837,14,18 PLT
9,7260878,15,18 PLT


In [213]:
toplam_palet['Araç Tipi'] = toplam_palet['Araç Tipi'].str.replace(' PLT', '')
toplam_palet.head(2)

,Sefer Numarası,Sefer_Toplam_Palet,Araç Tipi
0,7258421,15,18
1,7258425,18,18


In [214]:
toplam_palet['Araç Tipi'] = toplam_palet['Araç Tipi'].astype(int)
toplam_palet['Doluluk'] = toplam_palet['Sefer_Toplam_Palet'] / toplam_palet['Araç Tipi']

In [215]:
toplam_palet['Doluluk'].mean()

0.8810699588477364

In [216]:
toplam_palet.shape

(27, 4)

In [217]:
sefer_ozet_unq = sefer_ozet_unq[sefer_ozet_unq['Palet Sayısı'] != 0 ]

In [218]:
sefer_ozet_unq['Araç Tipi'].value_counts()

18 PLT    25
10 PLT     2
Name: Araç Tipi, dtype: int64

In [219]:
sefer_ozet_unq[sefer_ozet_unq['Araç Tipi'] == '33 PLT']

,Cari No,Sefer Numarası,Plaka,Araç Tipi,Rulot Sayısı,Palet Sayısı,Toplam Rulot ve Palet Sayısı,Malzeme Palet Sayısı,Toplanan Koli,Mağaza,Koli Sayısı,Değerli Kasa,Maliyet Tutar,Çıkış Tarihi,Dönüş Tarihi,Sevkiyat Süre Saat,Sefer Durumu,Mağaza Sayısı,Doluluk Oranı(%),Toplanan Hacim,Toplanan Ağırlık (kg),Rulot Kapasitesi,En Uzak Mağaza Km,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27


In [220]:
plaka_unq = sefer_ozet.drop_duplicates('Plaka')
plaka_unq = plaka_unq[plaka_unq['Palet Sayısı'] != 0 ]
plaka_unq['Araç Tipi'].value_counts()

18 PLT    13
10 PLT     2
Name: Araç Tipi, dtype: int64

In [221]:
sefer_ozet.groupby('Plaka')['Sefer Numarası'].nunique()

Plaka
01 BOJ 49     2
20 AAJ 084    1
20AAJ084      1
33 NG 653     1
33CFN11       2
34 EDM 477    3
34 ESL 981    2
34 ESM 048    3
34 GH 7474    2
34 HU 9124    2
34 HY 5365    2
34CFG216      2
34UG2185      1
38AAG327      2
59 EAE 552    1
Name: Sefer Numarası, dtype: int64

In [222]:
sef_oz_arac_sefer_unq = sefer_ozet.drop_duplicates(['Plaka','Sefer Numarası'], keep = 'first')[['Sefer Numarası', 'Plaka',
                                                                                      'Araç Tipi', 'En Uzak Mağaza Km']]
sef_oz_arac_sefer_unq.head()

,Sefer Numarası,Plaka,Araç Tipi,En Uzak Mağaza Km
2525,7258421,33CFN11,18 PLT,80.0
2528,7258425,34UG2185,18 PLT,65.0
2580,7260533,38AAG327,18 PLT,65.0
2583,7260581,34 HY 5365,18 PLT,65.0
2588,7260626,34 HU 9124,18 PLT,65.0


In [223]:
sefer_ozet.groupby('Plaka')['Araç Tipi'].value_counts()

Plaka       Araç Tipi
01 BOJ 49   18 PLT        6
20 AAJ 084  18 PLT        3
20AAJ084    18 PLT        3
33 NG 653   10 PLT        2
33CFN11     18 PLT        7
34 EDM 477  18 PLT       10
34 ESL 981  18 PLT        6
34 ESM 048  18 PLT       14
34 GH 7474  18 PLT       11
34 HU 9124  18 PLT        7
34 HY 5365  18 PLT        8
34CFG216    18 PLT        7
34UG2185    18 PLT        6
38AAG327    18 PLT       11
59 EAE 552  10 PLT        2
Name: Araç Tipi, dtype: int64

In [224]:
sefer_ozet.groupby('En Uzak Mağaza Km')['Plaka'].nunique()

En Uzak Mağaza Km
65.0     12
80.0      2
370.0     5
475.0     2
Name: Plaka, dtype: int64

In [225]:
sefer_ozet.groupby(['En Uzak Mağaza Km','Sefer Numarası'])['Plaka'].value_counts()

En Uzak Mağaza Km  Sefer Numarası  Plaka     
65.0               7258425         34UG2185      6
                   7260533         38AAG327      3
                   7260581         34 HY 5365    5
                   7260626         34 HU 9124    4
                   7260658         34 EDM 477    3
                   7260742         34 GH 7474    8
                   7260837         34CFG216      5
                   7260878         34 ESM 048    8
                   7260927         20 AAJ 084    3
                   7260930         34 ESL 981    3
                   7261203         34 EDM 477    5
                   7261313         34 ESM 048    3
                   7261392         38AAG327      8
                   7261466         59 EAE 552    2
                   7261749         01 BOJ 49     3
                   7261852         34 ESM 048    3
                   7261971         34 EDM 477    2
80.0               7258421         33CFN11       3
                   7260692         0

In [226]:
sefer_ozet.groupby(['En Uzak Mağaza Km','Sefer Numarası'])['Araç Tipi'].value_counts()

En Uzak Mağaza Km  Sefer Numarası  Araç Tipi
65.0               7258425         18 PLT       6
                   7260533         18 PLT       3
                   7260581         18 PLT       5
                   7260626         18 PLT       4
                   7260658         18 PLT       3
                   7260742         18 PLT       8
                   7260837         18 PLT       5
                   7260878         18 PLT       8
                   7260927         18 PLT       3
                   7260930         18 PLT       3
                   7261203         18 PLT       5
                   7261313         18 PLT       3
                   7261392         18 PLT       8
                   7261466         10 PLT       2
                   7261749         18 PLT       3
                   7261852         18 PLT       3
                   7261971         18 PLT       2
80.0               7258421         18 PLT       3
                   7260692         18 PLT       3
     

In [227]:
sefer_ozet.groupby('En Uzak Mağaza Km')['Araç Tipi'].unique()

En Uzak Mağaza Km
65.0     [18 PLT, 10 PLT]
80.0             [18 PLT]
370.0    [10 PLT, 18 PLT]
475.0            [18 PLT]
Name: Araç Tipi, dtype: object

In [228]:
sefer_ozet.groupby('En Uzak Mağaza Km')['Sefer Numarası'].nunique()

En Uzak Mağaza Km
65.0     17
80.0      3
370.0     5
475.0     2
Name: Sefer Numarası, dtype: int64

In [229]:
# (Aylık Kira Bedeli (10-18 PLT) / (Aylık Sefer Sayısı (10-18 PLT, En Uzak Km))) * 9.23 * 0.06 

In [230]:
sef_oz_arac_sefer_unq.loc[sef_oz_arac_sefer_unq['Araç Tipi'] == '18 PLT', 'amortisman'] = 9299
sef_oz_arac_sefer_unq.loc[sef_oz_arac_sefer_unq['Araç Tipi'] == '10 PLT', 'amortisman'] = 7531
sef_oz_arac_sefer_unq.loc[sef_oz_arac_sefer_unq['Araç Tipi'] == '33 PLT', 'amortisman'] = 19513
sef_oz_arac_sefer_unq.loc[sef_oz_arac_sefer_unq['Araç Tipi'] == '21 PLT', 'amortisman'] = 9590
sef_oz_arac_sefer_unq.head()

,Sefer Numarası,Plaka,Araç Tipi,En Uzak Mağaza Km,amortisman
2525,7258421,33CFN11,18 PLT,80.0,9299.0
2528,7258425,34UG2185,18 PLT,65.0,9299.0
2580,7260533,38AAG327,18 PLT,65.0,9299.0
2583,7260581,34 HY 5365,18 PLT,65.0,9299.0
2588,7260626,34 HU 9124,18 PLT,65.0,9299.0


In [231]:
aylik_sefer_sayisi = pd.DataFrame(data = {'baslangic_km': [0, 66, 151, 201, 401], 
                                          'bitis_km': [65, 150, 200, 400,10000],
                                          'plt_32': [39, 26, 26, 26, 20], 
                                          'plt_21': [47, 41, 35, 26, 20], 
                                          'plt_18': [47, 41, 35, 26, 20],
                                          'plt_12_15': [47, 41, 35, 26, 20],
                                          'plt_10': [47, 41, 35, 26, 20]})
aylik_sefer_sayisi.head()

,baslangic_km,bitis_km,plt_32,plt_21,plt_18,plt_12_15,plt_10
0,0,65,39,47,47,47,47
1,66,150,26,41,41,41,41
2,151,200,26,35,35,35,35
3,201,400,26,26,26,26,26
4,401,10000,20,20,20,20,20


In [232]:
sef_oz_arac_sefer_unq.loc[(sef_oz_arac_sefer_unq['En Uzak Mağaza Km']<=65) & (sef_oz_arac_sefer_unq['Araç Tipi']=='10 PLT'), 'aylik_sefer_sayi'] = 47
sef_oz_arac_sefer_unq.loc[(sef_oz_arac_sefer_unq['En Uzak Mağaza Km']<=65) & (sef_oz_arac_sefer_unq['Araç Tipi']=='18 PLT'), 'aylik_sefer_sayi'] = 47
sef_oz_arac_sefer_unq.loc[(sef_oz_arac_sefer_unq['En Uzak Mağaza Km']<=65) & (sef_oz_arac_sefer_unq['Araç Tipi']=='21 PLT'), 'aylik_sefer_sayi'] = 47
sef_oz_arac_sefer_unq.loc[(sef_oz_arac_sefer_unq['En Uzak Mağaza Km']<=65) & (sef_oz_arac_sefer_unq['Araç Tipi']=='33 PLT'), 'aylik_sefer_sayi'] = 39

sef_oz_arac_sefer_unq.loc[(sef_oz_arac_sefer_unq['En Uzak Mağaza Km']>65) & (sef_oz_arac_sefer_unq['En Uzak Mağaza Km']<=150) & (sef_oz_arac_sefer_unq['Araç Tipi']=='10 PLT'), 'aylik_sefer_sayi'] = 41
sef_oz_arac_sefer_unq.loc[(sef_oz_arac_sefer_unq['En Uzak Mağaza Km']>65) & (sef_oz_arac_sefer_unq['En Uzak Mağaza Km']<=150) & (sef_oz_arac_sefer_unq['Araç Tipi']=='18 PLT'), 'aylik_sefer_sayi'] = 41
sef_oz_arac_sefer_unq.loc[(sef_oz_arac_sefer_unq['En Uzak Mağaza Km']>65) & (sef_oz_arac_sefer_unq['En Uzak Mağaza Km']<=150) & (sef_oz_arac_sefer_unq['Araç Tipi']=='21 PLT'), 'aylik_sefer_sayi'] = 41
sef_oz_arac_sefer_unq.loc[(sef_oz_arac_sefer_unq['En Uzak Mağaza Km']>65) & (sef_oz_arac_sefer_unq['En Uzak Mağaza Km']<=150) & (sef_oz_arac_sefer_unq['Araç Tipi']=='33 PLT'), 'aylik_sefer_sayi'] = 26


sef_oz_arac_sefer_unq.loc[(sef_oz_arac_sefer_unq['En Uzak Mağaza Km']>150) & (sef_oz_arac_sefer_unq['En Uzak Mağaza Km']<=200) & (sef_oz_arac_sefer_unq['Araç Tipi']=='10 PLT'), 'aylik_sefer_sayi'] =35
sef_oz_arac_sefer_unq.loc[(sef_oz_arac_sefer_unq['En Uzak Mağaza Km']>150) & (sef_oz_arac_sefer_unq['En Uzak Mağaza Km']<=200) & (sef_oz_arac_sefer_unq['Araç Tipi']=='18 PLT'), 'aylik_sefer_sayi'] =35
sef_oz_arac_sefer_unq.loc[(sef_oz_arac_sefer_unq['En Uzak Mağaza Km']>150) & (sef_oz_arac_sefer_unq['En Uzak Mağaza Km']<=200) & (sef_oz_arac_sefer_unq['Araç Tipi']=='21 PLT'), 'aylik_sefer_sayi'] =35
sef_oz_arac_sefer_unq.loc[(sef_oz_arac_sefer_unq['En Uzak Mağaza Km']>150) & (sef_oz_arac_sefer_unq['En Uzak Mağaza Km']<=200) & (sef_oz_arac_sefer_unq['Araç Tipi']=='33 PLT'), 'aylik_sefer_sayi'] =26


sef_oz_arac_sefer_unq.loc[(sef_oz_arac_sefer_unq['En Uzak Mağaza Km']>200) & (sef_oz_arac_sefer_unq['En Uzak Mağaza Km']<=400) & (sef_oz_arac_sefer_unq['Araç Tipi']=='10 PLT'), 'aylik_sefer_sayi'] = 26
sef_oz_arac_sefer_unq.loc[(sef_oz_arac_sefer_unq['En Uzak Mağaza Km']>200) & (sef_oz_arac_sefer_unq['En Uzak Mağaza Km']<=400) & (sef_oz_arac_sefer_unq['Araç Tipi']=='18 PLT'), 'aylik_sefer_sayi'] = 26
sef_oz_arac_sefer_unq.loc[(sef_oz_arac_sefer_unq['En Uzak Mağaza Km']>200) & (sef_oz_arac_sefer_unq['En Uzak Mağaza Km']<=400) & (sef_oz_arac_sefer_unq['Araç Tipi']=='21 PLT'), 'aylik_sefer_sayi'] = 26
sef_oz_arac_sefer_unq.loc[(sef_oz_arac_sefer_unq['En Uzak Mağaza Km']>200) & (sef_oz_arac_sefer_unq['En Uzak Mağaza Km']<=400) & (sef_oz_arac_sefer_unq['Araç Tipi']=='33 PLT'), 'aylik_sefer_sayi'] = 26


sef_oz_arac_sefer_unq.loc[(sef_oz_arac_sefer_unq['En Uzak Mağaza Km']>400) & (sef_oz_arac_sefer_unq['Araç Tipi']=='10 PLT'), 'aylik_sefer_sayi'] = 20
sef_oz_arac_sefer_unq.loc[(sef_oz_arac_sefer_unq['En Uzak Mağaza Km']>400) & (sef_oz_arac_sefer_unq['Araç Tipi']=='18 PLT'), 'aylik_sefer_sayi'] = 20
sef_oz_arac_sefer_unq.loc[(sef_oz_arac_sefer_unq['En Uzak Mağaza Km']>400) & (sef_oz_arac_sefer_unq['Araç Tipi']=='21 PLT'), 'aylik_sefer_sayi'] = 20
sef_oz_arac_sefer_unq.loc[(sef_oz_arac_sefer_unq['En Uzak Mağaza Km']>400) & (sef_oz_arac_sefer_unq['Araç Tipi']=='33 PLT'), 'aylik_sefer_sayi'] = 20

sef_oz_arac_sefer_unq.head()

,Sefer Numarası,Plaka,Araç Tipi,En Uzak Mağaza Km,amortisman,aylik_sefer_sayi
2525,7258421,33CFN11,18 PLT,80.0,9299.0,41.0
2528,7258425,34UG2185,18 PLT,65.0,9299.0,47.0
2580,7260533,38AAG327,18 PLT,65.0,9299.0,47.0
2583,7260581,34 HY 5365,18 PLT,65.0,9299.0,47.0
2588,7260626,34 HU 9124,18 PLT,65.0,9299.0,47.0


In [233]:
sef_oz_arac_sefer_unq.loc[(sef_oz_arac_sefer_unq['Araç Tipi']=='10 PLT'), 'yakma_orani'] = 0.23
sef_oz_arac_sefer_unq.loc[(sef_oz_arac_sefer_unq['Araç Tipi']=='18 PLT'), 'yakma_orani'] = 0.3
sef_oz_arac_sefer_unq.loc[(sef_oz_arac_sefer_unq['Araç Tipi']=='21 PLT'), 'yakma_orani'] = 0.31
sef_oz_arac_sefer_unq.loc[(sef_oz_arac_sefer_unq['Araç Tipi']=='33 PLT'), 'yakma_orani'] = 0.37

In [234]:
# (Aylık Kira Bedeli (10-18 PLT) / (Aylık Sefer Sayısı (10-18 PLT, En Uzak Km))) * 9.23 * 0.06 

In [235]:
sef_oz_arac_sefer_unq['maliyet'] = ((sef_oz_arac_sefer_unq['amortisman'] / sef_oz_arac_sefer_unq['aylik_sefer_sayi']) +\
                          (sef_oz_arac_sefer_unq['En Uzak Mağaza Km'] * sef_oz_arac_sefer_unq['yakma_orani'] * 9.23) ) * 1.06  
sef_oz_arac_sefer_unq.tail(10)

,Sefer Numarası,Plaka,Araç Tipi,En Uzak Mağaza Km,amortisman,aylik_sefer_sayi,yakma_orani,maliyet
2642,7261391,34 GH 7474,18 PLT,370.0,9299.0,26.0,0.30,1465.114877
2645,7261392,38AAG327,18 PLT,65.0,9299.0,47.0,0.30,400.506228
2653,7261450,33CFN11,18 PLT,80.0,9299.0,41.0,0.30,475.224371
2657,7261466,59 EAE 552,10 PLT,65.0,7531.0,47.0,0.23,316.115895
2659,7261539,34 HY 5365,18 PLT,370.0,9299.0,26.0,0.30,1465.114877
2662,7261551,34CFG216,18 PLT,370.0,9299.0,26.0,0.30,1465.114877
2664,7261636,34 HU 9124,18 PLT,370.0,9299.0,26.0,0.30,1465.114877
2667,7261749,01 BOJ 49,18 PLT,65.0,9299.0,47.0,0.30,400.506228
2670,7261852,34 ESM 048,18 PLT,65.0,9299.0,47.0,0.30,400.506228
2673,7261971,34 EDM 477,18 PLT,65.0,9299.0,47.0,0.30,400.506228


In [236]:
# (gunluk_kiralama_bedeli/unq_plaka_count) bir kere
# + her bir sefer icin (sef_oz_arac_sefer_unq['En Uzak Mağaza Km'] * sef_oz_arac_sefer_unq['yakma_orani'] * 9.23)  * 1.06)

In [237]:
sef_oz_arac_sefer_unq.shape

(27, 8)

In [238]:
sef_oz_arac_sefer_unq.maliyet.sum()

18924.05961447008

In [239]:
sef_oz_arac_sefer_unq.shape

(27, 8)

### Model Maliyet

In [240]:
mdl_sefer_unq = output4.drop_duplicates('sefer_id', keep = 'first')
mdl_sefer_unq['max_distance'].value_counts()

363.6974    1
50.4008     1
366.9316    1
50.0288     1
60.5348     1
64.8668     1
53.0806     1
64.7606     1
64.7892     1
59.7714     1
56.6282     1
68.6746     1
65.9618     1
73.8398     1
366.6504    1
48.0778     1
82.0598     1
69.3092     1
355.4946    1
362.7344    1
355.6722    1
45.7610     1
365.9738    1
41.4390     1
41.9196     1
46.6784     1
Name: max_distance, dtype: int64

In [241]:
# rotalar2 = sefer_unq.copy()

In [242]:
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance'] <= 65),'max_distance'] = 65
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance'] > 65) & (mdl_sefer_unq['max_distance'] <= 80),'max_distance'] = 80
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance'] > 80) & (mdl_sefer_unq['max_distance'] <= 105),'max_distance'] = 105
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance'] > 105) & (mdl_sefer_unq['max_distance'] <= 120),'max_distance'] = 120
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance'] > 120) & (mdl_sefer_unq['max_distance'] <= 125),'max_distance'] = 125
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance'] > 125) & (mdl_sefer_unq['max_distance'] <= 136),'max_distance'] = 136
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance'] > 136) & (mdl_sefer_unq['max_distance'] <= 150),'max_distance'] = 150
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance'] > 150) & (mdl_sefer_unq['max_distance'] <= 155),'max_distance'] = 155
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance'] > 155) & (mdl_sefer_unq['max_distance'] <= 160),'max_distance'] = 160
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance'] > 160) & (mdl_sefer_unq['max_distance'] <= 176),'max_distance'] = 176
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance'] > 176) & (mdl_sefer_unq['max_distance'] <= 178),'max_distance'] = 178
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance'] > 178) & (mdl_sefer_unq['max_distance'] <= 194),'max_distance'] = 194
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance'] > 194) & (mdl_sefer_unq['max_distance'] <= 200),'max_distance'] = 200
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance'] > 200) & (mdl_sefer_unq['max_distance'] <= 226),'max_distance'] = 226
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance'] > 226) & (mdl_sefer_unq['max_distance'] <= 244),'max_distance'] = 244
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance'] > 244) & (mdl_sefer_unq['max_distance'] <= 255),'max_distance'] = 255
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance'] > 255) & (mdl_sefer_unq['max_distance'] <= 330),'max_distance'] = 330
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance'] > 330) & (mdl_sefer_unq['max_distance'] <= 370),'max_distance'] = 370
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance'] > 370) & (mdl_sefer_unq['max_distance'] <= 375),'max_distance'] = 375
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance'] > 375) & (mdl_sefer_unq['max_distance'] <= 380),'max_distance'] = 380
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance'] > 380) & (mdl_sefer_unq['max_distance'] <= 460),'max_distance'] = 460
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance'] > 460) & (mdl_sefer_unq['max_distance'] <= 475),'max_distance'] = 475
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance'] > 475) & (mdl_sefer_unq['max_distance'] <= 480),'max_distance'] = 480
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance'] > 480) & (mdl_sefer_unq['max_distance'] <= 490),'max_distance'] = 490

In [243]:
mdl_sefer_unq['max_distance'].value_counts()

65.0     14
370.0     7
80.0      4
105.0     1
Name: max_distance, dtype: int64

In [244]:
mdl_sefer_unq['capacities'].value_counts()

18    24
10     2
Name: capacities, dtype: int64

In [245]:
mdl_sefer_unq

,arac_id,capacities,routes,birim_list,magaza_list,palet_list,dist_list,sefer_id,max_distance
0,0,10,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,1.0,370.0
3,1,10,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,2.0,65.0
7,2,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,3.0,65.0
11,2,18,0.0,99940.0,MERSIN DAGITIM MERKEZI,0,0.0,4.0,65.0
15,3,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,5.0,370.0
21,4,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,6.0,65.0
31,5,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,7.0,370.0
36,6,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,8.0,370.0
40,7,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,9.0,370.0
45,8,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,10.0,80.0


In [246]:
mdl_sefer_unq.loc[mdl_sefer_unq['capacities'] == 18, 'amortisman'] = 9299
mdl_sefer_unq.loc[mdl_sefer_unq['capacities'] == 10, 'amortisman'] = 7531
mdl_sefer_unq.loc[mdl_sefer_unq['capacities'] == 33, 'amortisman'] = 19513
mdl_sefer_unq.loc[mdl_sefer_unq['capacities'] == 21, 'amortisman'] = 9590
mdl_sefer_unq.head()

,arac_id,capacities,routes,birim_list,magaza_list,palet_list,dist_list,sefer_id,max_distance,amortisman
0,0,10,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,1.0,370.0,7531.0
3,1,10,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,2.0,65.0,7531.0
7,2,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,3.0,65.0,9299.0
11,2,18,0.0,99940.0,MERSIN DAGITIM MERKEZI,0,0.0,4.0,65.0,9299.0
15,3,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,5.0,370.0,9299.0


In [247]:
aylik_sefer_sayisi = pd.DataFrame(data = {'baslangic_km': [0, 66, 151, 201, 401], 
                                          'bitis_km': [65, 150, 200, 400,10000],
                                          'plt_32': [39, 26, 26, 26, 20], 
                                          'plt_21': [47, 41, 35, 26, 20], 
                                          'plt_18': [47, 41, 35, 26, 20],
                                          'plt_12_15': [47, 41, 35, 26, 20],
                                          'plt_10': [47, 41, 35, 26, 20]})
aylik_sefer_sayisi.head()

,baslangic_km,bitis_km,plt_32,plt_21,plt_18,plt_12_15,plt_10
0,0,65,39,47,47,47,47
1,66,150,26,41,41,41,41
2,151,200,26,35,35,35,35
3,201,400,26,26,26,26,26
4,401,10000,20,20,20,20,20


In [248]:
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance']<=65) & (mdl_sefer_unq['capacities']==10), 'aylik_sefer_sayi'] = 47
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance']<=65) & (mdl_sefer_unq['capacities']==18), 'aylik_sefer_sayi'] = 47
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance']<=65) & (mdl_sefer_unq['capacities']==21), 'aylik_sefer_sayi'] = 47
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance']<=65) & (mdl_sefer_unq['capacities']==33), 'aylik_sefer_sayi'] = 39

mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance']>65) & (mdl_sefer_unq['max_distance']<=150) & (mdl_sefer_unq['capacities']==10), 'aylik_sefer_sayi'] = 41
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance']>65) & (mdl_sefer_unq['max_distance']<=150) & (mdl_sefer_unq['capacities']==18), 'aylik_sefer_sayi'] = 41
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance']>65) & (mdl_sefer_unq['max_distance']<=150) & (mdl_sefer_unq['capacities']==21), 'aylik_sefer_sayi'] = 41
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance']>65) & (mdl_sefer_unq['max_distance']<=150) & (mdl_sefer_unq['capacities']==33), 'aylik_sefer_sayi'] = 26


mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance']>150) & (mdl_sefer_unq['max_distance']<=200) & (mdl_sefer_unq['capacities']==10), 'aylik_sefer_sayi'] =35
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance']>150) & (mdl_sefer_unq['max_distance']<=200) & (mdl_sefer_unq['capacities']==18), 'aylik_sefer_sayi'] =35
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance']>150) & (mdl_sefer_unq['max_distance']<=200) & (mdl_sefer_unq['capacities']==21), 'aylik_sefer_sayi'] =35
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance']>150) & (mdl_sefer_unq['max_distance']<=200) & (mdl_sefer_unq['capacities']==33), 'aylik_sefer_sayi'] =26


mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance']>200) & (mdl_sefer_unq['max_distance']<=400) & (mdl_sefer_unq['capacities']==10), 'aylik_sefer_sayi'] = 26
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance']>200) & (mdl_sefer_unq['max_distance']<=400) & (mdl_sefer_unq['capacities']==18), 'aylik_sefer_sayi'] = 26
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance']>200) & (mdl_sefer_unq['max_distance']<=400) & (mdl_sefer_unq['capacities']==21), 'aylik_sefer_sayi'] = 26
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance']>200) & (mdl_sefer_unq['max_distance']<=400) & (mdl_sefer_unq['capacities']==33), 'aylik_sefer_sayi'] = 26


mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance']>400) & (mdl_sefer_unq['capacities']==10), 'aylik_sefer_sayi'] = 20
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance']>400) & (mdl_sefer_unq['capacities']==18), 'aylik_sefer_sayi'] = 20
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance']>400) & (mdl_sefer_unq['capacities']==21), 'aylik_sefer_sayi'] = 20
mdl_sefer_unq.loc[(mdl_sefer_unq['max_distance']>400) & (mdl_sefer_unq['capacities']==33), 'aylik_sefer_sayi'] = 20

mdl_sefer_unq.head()

,arac_id,capacities,routes,birim_list,magaza_list,palet_list,dist_list,sefer_id,max_distance,amortisman,aylik_sefer_sayi
0,0,10,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,1.0,370.0,7531.0,26.0
3,1,10,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,2.0,65.0,7531.0,47.0
7,2,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,3.0,65.0,9299.0,47.0
11,2,18,0.0,99940.0,MERSIN DAGITIM MERKEZI,0,0.0,4.0,65.0,9299.0,47.0
15,3,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,5.0,370.0,9299.0,26.0


In [249]:
mdl_sefer_unq['capacities'].value_counts()

18    24
10     2
Name: capacities, dtype: int64

In [250]:
mdl_sefer_unq.loc[(mdl_sefer_unq['capacities'] == 10), 'yakma_orani'] = 0.23
mdl_sefer_unq.loc[(mdl_sefer_unq['capacities'] == 18), 'yakma_orani'] = 0.3
mdl_sefer_unq.loc[(mdl_sefer_unq['capacities'] == 21), 'yakma_orani'] = 0.31
mdl_sefer_unq.loc[(mdl_sefer_unq['capacities'] == 33), 'yakma_orani'] = 0.37

In [251]:
# (Aylık Kira Bedeli (10-18 PLT) / (Aylık Sefer Sayısı (10-18 PLT, En Uzak Km))) * 9.23 * 0.06 

In [252]:
mdl_sefer_unq['maliyet'] = ((mdl_sefer_unq['amortisman'] / mdl_sefer_unq['aylik_sefer_sayi']) +\
                          (mdl_sefer_unq['max_distance'] * mdl_sefer_unq['yakma_orani'] * 9.23) ) * 1.06  
mdl_sefer_unq.tail(10)

,arac_id,capacities,routes,birim_list,magaza_list,palet_list,dist_list,sefer_id,max_distance,amortisman,aylik_sefer_sayi,yakma_orani,maliyet
73,10,18,0.0,99946.0,MERSIN DAGITIM MERKEZI,0,0.0,17.0,65.0,9299.0,47.0,0.3,400.506228
77,11,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,18.0,65.0,9299.0,47.0,0.3,400.506228
80,12,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,19.0,65.0,9299.0,47.0,0.3,400.506228
84,12,18,0.0,99944.0,MERSIN DAGITIM MERKEZI,0,0.0,20.0,65.0,9299.0,47.0,0.3,400.506228
89,13,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,21.0,65.0,9299.0,47.0,0.3,400.506228
94,13,18,0.0,99943.0,MERSIN DAGITIM MERKEZI,0,0.0,22.0,65.0,9299.0,47.0,0.3,400.506228
100,14,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,23.0,65.0,9299.0,47.0,0.3,400.506228
107,14,18,0.0,99942.0,MERSIN DAGITIM MERKEZI,0,0.0,24.0,65.0,9299.0,47.0,0.3,400.506228
117,15,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,25.0,370.0,9299.0,26.0,0.3,1465.114877
122,15,18,0.0,99939.0,MERSIN DAGITIM MERKEZI,0,0.0,26.0,65.0,9299.0,47.0,0.3,400.506228


### Model & Gercek Karsilastirma

#### Toplam Araç Sayısı

In [253]:
# Model 
output4.arac_id.nunique()

16

In [254]:
# Gercek
sefer_ozet['Plaka'].nunique()

15

#### Unique Araç Tipleri

In [255]:
# Model 
mdl_arac_unq = output4.drop_duplicates('arac_id', keep = 'first')
mdl_arac_unq.capacities.value_counts()

18    14
10     2
Name: capacities, dtype: int64

In [256]:
# Gercek
sfr_arac_unq = sef_oz_arac_sefer_unq.drop_duplicates('Plaka', keep = 'first')
sfr_arac_unq['Araç Tipi'].value_counts()

18 PLT    13
10 PLT     2
Name: Araç Tipi, dtype: int64

#### Seferlerde Kullanılan Araç Tipleri

In [257]:
mdl_sefer_unq.capacities.value_counts()

18    24
10     2
Name: capacities, dtype: int64

In [258]:
sef_oz_arac_sefer_unq['Araç Tipi'].value_counts()

18 PLT    25
10 PLT     2
Name: Araç Tipi, dtype: int64

#### Mesafelere Göre Araç Tipleri

In [259]:
mdl_sefer_unq.loc[(mdl_sefer_unq['sefer_id'] == 25) & (mdl_sefer_unq['max_distance'] == 370), 'max_distance'] = 475
mdl_sefer_unq.loc[(mdl_sefer_unq['sefer_id'] == 25) & (mdl_sefer_unq['max_distance'] == 475), 'aylik_sefer_sayi'] = 26
mdl_sefer_unq.loc[(mdl_sefer_unq['sefer_id'] == 25) & (mdl_sefer_unq['max_distance'] == 475), 'maliyet'] = 1887.438973

In [260]:
mdl_sefer_unq

,arac_id,capacities,routes,birim_list,magaza_list,palet_list,dist_list,sefer_id,max_distance,amortisman,aylik_sefer_sayi,yakma_orani,maliyet
0,0,10,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,1.0,370.0,7531.0,26.0,0.23,1139.634457
3,1,10,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,2.0,65.0,7531.0,47.0,0.23,316.115895
7,2,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,3.0,65.0,9299.0,47.0,0.30,400.506228
11,2,18,0.0,99940.0,MERSIN DAGITIM MERKEZI,0,0.0,4.0,65.0,9299.0,47.0,0.30,400.506228
15,3,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,5.0,370.0,9299.0,26.0,0.30,1465.114877
21,4,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,6.0,65.0,9299.0,47.0,0.30,400.506228
31,5,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,7.0,370.0,9299.0,26.0,0.30,1465.114877
36,6,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,8.0,370.0,9299.0,26.0,0.30,1465.114877
40,7,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,9.0,370.0,9299.0,26.0,0.30,1465.114877
45,8,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,10.0,80.0,9299.0,41.0,0.30,475.224371


In [261]:
# Model
mdl_sefer_unq['max_distance'].value_counts()

65.0     14
370.0     6
80.0      4
105.0     1
475.0     1
Name: max_distance, dtype: int64

In [262]:
65*14 + 370*6 + 80*4 + 105 + 475

4030

In [263]:
# Gercek
sfr_sefer_unq = sef_oz_arac_sefer_unq.drop_duplicates('Sefer Numarası', keep = 'first')
sfr_sefer_unq.loc[sfr_sefer_unq['Sefer Numarası'] == 7261300, 'En Uzak Mağaza Km'] = 370
sfr_sefer_unq.loc[sfr_sefer_unq['Sefer Numarası'] == 7261300, 'aylik_sefer_sayi'] = 26
sfr_sefer_unq.loc[sfr_sefer_unq['Sefer Numarası'] == 7261300, 'maliyet'] = 1465.426913
sfr_sefer_unq['En Uzak Mağaza Km'].value_counts()

65.0     17
370.0     6
80.0      3
475.0     1
Name: En Uzak Mağaza Km, dtype: int64

In [264]:
sfr_sefer_unq.shape, sef_oz_arac_sefer_unq.shape

((27, 8), (27, 8))

In [265]:
sfr_sefer_unq

,Sefer Numarası,Plaka,Araç Tipi,En Uzak Mağaza Km,amortisman,aylik_sefer_sayi,yakma_orani,maliyet
2525,7258421,33CFN11,18 PLT,80.0,9299.0,41.0,0.30,475.224371
2528,7258425,34UG2185,18 PLT,65.0,9299.0,47.0,0.30,400.506228
2580,7260533,38AAG327,18 PLT,65.0,9299.0,47.0,0.30,400.506228
2583,7260581,34 HY 5365,18 PLT,65.0,9299.0,47.0,0.30,400.506228
2588,7260626,34 HU 9124,18 PLT,65.0,9299.0,47.0,0.30,400.506228
2592,7260658,34 EDM 477,18 PLT,65.0,9299.0,47.0,0.30,400.506228
2595,7260692,01 BOJ 49,18 PLT,80.0,9299.0,41.0,0.30,475.224371
2598,7260742,34 GH 7474,18 PLT,65.0,9299.0,47.0,0.30,400.506228
2606,7260837,34CFG216,18 PLT,65.0,9299.0,47.0,0.30,400.506228
2611,7260878,34 ESM 048,18 PLT,65.0,9299.0,47.0,0.30,400.506228


#### En Uzak Mesafe Km Toplam

In [266]:
mdl_sefer_unq['max_distance'].sum()

4030.0

In [267]:
sfr_sefer_unq['En Uzak Mağaza Km'].sum()

4040.0

#### Maliyet

In [268]:
# Model
mdl_sefer_unq.maliyet.sum(), mdl_sefer_unq.maliyet.shape

(18324.84502287785, (26,))

In [269]:
# Gercek
sfr_sefer_unq.maliyet.sum(), sfr_sefer_unq.maliyet.shape

(18502.44802747008, (27,))

In [270]:
sfr_sefer_unq

,Sefer Numarası,Plaka,Araç Tipi,En Uzak Mağaza Km,amortisman,aylik_sefer_sayi,yakma_orani,maliyet
2525,7258421,33CFN11,18 PLT,80.0,9299.0,41.0,0.30,475.224371
2528,7258425,34UG2185,18 PLT,65.0,9299.0,47.0,0.30,400.506228
2580,7260533,38AAG327,18 PLT,65.0,9299.0,47.0,0.30,400.506228
2583,7260581,34 HY 5365,18 PLT,65.0,9299.0,47.0,0.30,400.506228
2588,7260626,34 HU 9124,18 PLT,65.0,9299.0,47.0,0.30,400.506228
2592,7260658,34 EDM 477,18 PLT,65.0,9299.0,47.0,0.30,400.506228
2595,7260692,01 BOJ 49,18 PLT,80.0,9299.0,41.0,0.30,475.224371
2598,7260742,34 GH 7474,18 PLT,65.0,9299.0,47.0,0.30,400.506228
2606,7260837,34CFG216,18 PLT,65.0,9299.0,47.0,0.30,400.506228
2611,7260878,34 ESM 048,18 PLT,65.0,9299.0,47.0,0.30,400.506228


In [271]:
# Gunluk Iyilestirme
(sfr_sefer_unq.maliyet.sum() - mdl_sefer_unq.maliyet.sum()) / sfr_sefer_unq.maliyet.sum()

0.009598892229209259

In [272]:
# Aylik Iyilestirme
30 * (sfr_sefer_unq.maliyet.sum() - mdl_sefer_unq.maliyet.sum()) / sfr_sefer_unq.maliyet.sum()

0.28796676687627776

#### Araç Doluluğu

In [273]:
sefer_plt_toplam = output4.groupby('sefer_id')['palet_list'].sum().reset_index()
sefer_plt_toplam = sefer_plt_toplam[sefer_plt_toplam['palet_list'] != 0]
# output4_unq = output4.drop_duplicates('sefer_id',keep = 'first')
sefer_plt_toplam = sefer_plt_toplam.merge(mdl_sefer_unq[['sefer_id','capacities']], on = 'sefer_id', how = 'left')
sefer_plt_toplam['Doluluk'] = sefer_plt_toplam['palet_list'] / sefer_plt_toplam['capacities']
sefer_plt_toplam

,sefer_id,palet_list,capacities,Doluluk
0,1.0,8,10,0.8
1,2.0,10,10,1.0
2,3.0,18,18,1.0
3,4.0,15,18,0.833333
4,5.0,17,18,0.944444
5,6.0,17,18,0.944444
6,7.0,18,18,1.0
7,8.0,17,18,0.944444
8,9.0,18,18,1.0
9,10.0,18,18,1.0


In [274]:
 output4

,arac_id,capacities,routes,birim_list,magaza_list,palet_list,dist_list,sefer_id,max_distance
0,0,10,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,1.0,363.6974
1,0,10,143.0,8167.0,ŞOK-NİGDE SELÇUK,7,363.6974,1.0,363.6974
2,0,10,117.0,8012.0,ŞOK-MERSİN TARSUS GAZİPAŞA,1,37.6158,1.0,363.6974
3,1,10,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,2.0,50.4008
4,1,10,88.0,5970.0,ŞOK-MERSİN KURDALİ,1,43.3244,2.0,50.4008
5,1,10,69.0,6632.0,ŞOK-MERSİN BAHÇELİEVLER,6,50.4008,2.0,50.4008
6,1,10,136.0,1100.0,ŞOK-MERSİN ÇAMLIBEL,3,44.8206,2.0,50.4008
7,2,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,3.0,41.9196
8,2,18,111.0,9812.0,ŞOK-MERSİN SİTELER,5,39.5564,3.0,41.9196
9,2,18,131.0,2032.0,ŞOK-MERSİN YENİMAHALLE,7,41.1586,3.0,41.9196


In [275]:
out370 = output4[ output4.max_distance > 340]
out370.groupby('sefer_id')['palet_list'].sum().reset_index()

,sefer_id,palet_list
0,1.0,8
1,5.0,17
2,7.0,18
3,8.0,17
4,9.0,18
5,13.0,18
6,25.0,18


In [276]:
mdl_plt_toplam = output4.groupby('sefer_id')['palet_list'].sum().reset_index()
mdl_plt_toplam

,sefer_id,palet_list
0,1.0,8
1,2.0,10
2,3.0,18
3,4.0,15
4,5.0,17
5,6.0,17
6,7.0,18
7,8.0,17
8,9.0,18
9,10.0,18


In [277]:
sefer_plt_toplam['Doluluk'].mean()

0.9132478632478632

In [278]:
mdl_sefer_unq

,arac_id,capacities,routes,birim_list,magaza_list,palet_list,dist_list,sefer_id,max_distance,amortisman,aylik_sefer_sayi,yakma_orani,maliyet
0,0,10,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,1.0,370.0,7531.0,26.0,0.23,1139.634457
3,1,10,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,2.0,65.0,7531.0,47.0,0.23,316.115895
7,2,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,3.0,65.0,9299.0,47.0,0.30,400.506228
11,2,18,0.0,99940.0,MERSIN DAGITIM MERKEZI,0,0.0,4.0,65.0,9299.0,47.0,0.30,400.506228
15,3,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,5.0,370.0,9299.0,26.0,0.30,1465.114877
21,4,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,6.0,65.0,9299.0,47.0,0.30,400.506228
31,5,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,7.0,370.0,9299.0,26.0,0.30,1465.114877
36,6,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,8.0,370.0,9299.0,26.0,0.30,1465.114877
40,7,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,9.0,370.0,9299.0,26.0,0.30,1465.114877
45,8,18,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,10.0,80.0,9299.0,41.0,0.30,475.224371


## SON SON SON

In [279]:
output5 = output4.merge( mdl_sefer_unq[['sefer_id','maliyet']], how = 'left', on = 'sefer_id')
output5.head()

,arac_id,capacities,routes,birim_list,magaza_list,palet_list,dist_list,sefer_id,max_distance,maliyet
0,0,10,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,1.0,363.6974,1139.634457
1,0,10,143.0,8167.0,ŞOK-NİGDE SELÇUK,7,363.6974,1.0,363.6974,1139.634457
2,0,10,117.0,8012.0,ŞOK-MERSİN TARSUS GAZİPAŞA,1,37.6158,1.0,363.6974,1139.634457
3,1,10,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,2.0,50.4008,316.115895
4,1,10,88.0,5970.0,ŞOK-MERSİN KURDALİ,1,43.3244,2.0,50.4008,316.115895


In [280]:
output5['oran'] = (output5['maliyet'] / output5['capacities']) * output5['palet_list']

In [281]:
output5.head()

,arac_id,capacities,routes,birim_list,magaza_list,palet_list,dist_list,sefer_id,max_distance,maliyet,oran
0,0,10,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,1.0,363.6974,1139.634457,0.0
1,0,10,143.0,8167.0,ŞOK-NİGDE SELÇUK,7,363.6974,1.0,363.6974,1139.634457,797.74412
2,0,10,117.0,8012.0,ŞOK-MERSİN TARSUS GAZİPAŞA,1,37.6158,1.0,363.6974,1139.634457,113.963446
3,1,10,0.0,99999.0,MERSIN DAGITIM MERKEZI,0,0.0,2.0,50.4008,316.115895,0.0
4,1,10,88.0,5970.0,ŞOK-MERSİN KURDALİ,1,43.3244,2.0,50.4008,316.115895,31.61159


In [282]:
output5 = output5[output5['magaza_list'] != 'MERSIN DAGITIM MERKEZI']
output5.head(2)

,arac_id,capacities,routes,birim_list,magaza_list,palet_list,dist_list,sefer_id,max_distance,maliyet,oran
1,0,10,143.0,8167.0,ŞOK-NİGDE SELÇUK,7,363.6974,1.0,363.6974,1139.634457,797.74412
2,0,10,117.0,8012.0,ŞOK-MERSİN TARSUS GAZİPAŞA,1,37.6158,1.0,363.6974,1139.634457,113.963446


In [283]:
output5['oran'].mean()

166.5166453623919

In [284]:
output5.shape

(103, 11)

In [285]:
sefer_ozet.head()

,Cari No,Sefer Numarası,Plaka,Araç Tipi,Rulot Sayısı,Palet Sayısı,Toplam Rulot ve Palet Sayısı,Malzeme Palet Sayısı,Toplanan Koli,Mağaza,Koli Sayısı,Değerli Kasa,Maliyet Tutar,Çıkış Tarihi,Dönüş Tarihi,Sevkiyat Süre Saat,Sefer Durumu,Mağaza Sayısı,Doluluk Oranı(%),Toplanan Hacim,Toplanan Ağırlık (kg),Rulot Kapasitesi,En Uzak Mağaza Km,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
2525,251062,7258421,33CFN11,18 PLT,0,7,8,0,558,MERSİN 34.CADDE,412.78,1,54454.1623,2021-12-17 00:58:10,2021-12-17 12:46:15,11 sa 48 dk,TAMAMLANDI,3,57,99361.0,317,30,80.0,NaN,NaN,NaN,NaN,NaN
2526,251062,7258421,33CFN11,18 PLT,0,7,8,0,694,MERSİN KAMPÜS,534.11,1,41757.1641,2021-12-17 00:58:10,2021-12-17 12:46:15,11 sa 48 dk,TAMAMLANDI,3,57,105192.0,356,30,80.0,NaN,NaN,NaN,NaN,NaN
2527,251062,7258421,33CFN11,18 PLT,0,1,1,0,109,MERSİN PARİS,40.81,0,40884.8421,2021-12-17 00:58:10,2021-12-17 12:46:15,11 sa 48 dk,TAMAMLANDI,3,57,3200.0,22,30,80.0,NaN,NaN,NaN,NaN,NaN
2528,251062,7258425,34UG2185,18 PLT,0,5,7,0,434,MERSİN TARSUS 100.YIL,306.07,1,26905.8693,2021-12-17 01:33:56,2021-12-17 11:59:54,10 sa 26 dk,TAMAMLANDI,6,96,60195.0,251,23,65.0,NaN,NaN,NaN,NaN,NaN
2529,251062,7258425,34UG2185,18 PLT,0,1,1,0,185,MERSİN TARSUS AKŞEMSETTİN,67.04,0,30939.4226,2021-12-17 01:33:56,2021-12-17 11:59:54,10 sa 26 dk,TAMAMLANDI,6,96,862.0,40,23,65.0,NaN,NaN,NaN,NaN,NaN


In [286]:
sefer_ozet.loc[sefer_ozet['Araç Tipi'] == '18 PLT', 'capacities'] = 18  
sefer_ozet.loc[sefer_ozet['Araç Tipi'] == '10 PLT', 'capacities'] = 10  

In [287]:
sefer_ozet.head(2)

,Cari No,Sefer Numarası,Plaka,Araç Tipi,Rulot Sayısı,Palet Sayısı,Toplam Rulot ve Palet Sayısı,Malzeme Palet Sayısı,Toplanan Koli,Mağaza,Koli Sayısı,Değerli Kasa,Maliyet Tutar,Çıkış Tarihi,Dönüş Tarihi,Sevkiyat Süre Saat,Sefer Durumu,Mağaza Sayısı,Doluluk Oranı(%),Toplanan Hacim,Toplanan Ağırlık (kg),Rulot Kapasitesi,En Uzak Mağaza Km,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,capacities
2525,251062,7258421,33CFN11,18 PLT,0,7,8,0,558,MERSİN 34.CADDE,412.78,1,54454.1623,2021-12-17 00:58:10,2021-12-17 12:46:15,11 sa 48 dk,TAMAMLANDI,3,57,99361.0,317,30,80.0,NaN,NaN,NaN,NaN,NaN,18.0
2526,251062,7258421,33CFN11,18 PLT,0,7,8,0,694,MERSİN KAMPÜS,534.11,1,41757.1641,2021-12-17 00:58:10,2021-12-17 12:46:15,11 sa 48 dk,TAMAMLANDI,3,57,105192.0,356,30,80.0,NaN,NaN,NaN,NaN,NaN,18.0


In [288]:
sefer_ozet.shape

(103, 29)

In [289]:
sef_oz_arac_sefer_unq.head()

,Sefer Numarası,Plaka,Araç Tipi,En Uzak Mağaza Km,amortisman,aylik_sefer_sayi,yakma_orani,maliyet
2525,7258421,33CFN11,18 PLT,80.0,9299.0,41.0,0.3,475.224371
2528,7258425,34UG2185,18 PLT,65.0,9299.0,47.0,0.3,400.506228
2580,7260533,38AAG327,18 PLT,65.0,9299.0,47.0,0.3,400.506228
2583,7260581,34 HY 5365,18 PLT,65.0,9299.0,47.0,0.3,400.506228
2588,7260626,34 HU 9124,18 PLT,65.0,9299.0,47.0,0.3,400.506228


In [290]:
sefer_ozet.head()

,Cari No,Sefer Numarası,Plaka,Araç Tipi,Rulot Sayısı,Palet Sayısı,Toplam Rulot ve Palet Sayısı,Malzeme Palet Sayısı,Toplanan Koli,Mağaza,Koli Sayısı,Değerli Kasa,Maliyet Tutar,Çıkış Tarihi,Dönüş Tarihi,Sevkiyat Süre Saat,Sefer Durumu,Mağaza Sayısı,Doluluk Oranı(%),Toplanan Hacim,Toplanan Ağırlık (kg),Rulot Kapasitesi,En Uzak Mağaza Km,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,capacities
2525,251062,7258421,33CFN11,18 PLT,0,7,8,0,558,MERSİN 34.CADDE,412.78,1,54454.1623,2021-12-17 00:58:10,2021-12-17 12:46:15,11 sa 48 dk,TAMAMLANDI,3,57,99361.0,317,30,80.0,NaN,NaN,NaN,NaN,NaN,18.0
2526,251062,7258421,33CFN11,18 PLT,0,7,8,0,694,MERSİN KAMPÜS,534.11,1,41757.1641,2021-12-17 00:58:10,2021-12-17 12:46:15,11 sa 48 dk,TAMAMLANDI,3,57,105192.0,356,30,80.0,NaN,NaN,NaN,NaN,NaN,18.0
2527,251062,7258421,33CFN11,18 PLT,0,1,1,0,109,MERSİN PARİS,40.81,0,40884.8421,2021-12-17 00:58:10,2021-12-17 12:46:15,11 sa 48 dk,TAMAMLANDI,3,57,3200.0,22,30,80.0,NaN,NaN,NaN,NaN,NaN,18.0
2528,251062,7258425,34UG2185,18 PLT,0,5,7,0,434,MERSİN TARSUS 100.YIL,306.07,1,26905.8693,2021-12-17 01:33:56,2021-12-17 11:59:54,10 sa 26 dk,TAMAMLANDI,6,96,60195.0,251,23,65.0,NaN,NaN,NaN,NaN,NaN,18.0
2529,251062,7258425,34UG2185,18 PLT,0,1,1,0,185,MERSİN TARSUS AKŞEMSETTİN,67.04,0,30939.4226,2021-12-17 01:33:56,2021-12-17 11:59:54,10 sa 26 dk,TAMAMLANDI,6,96,862.0,40,23,65.0,NaN,NaN,NaN,NaN,NaN,18.0


In [291]:
sefer_ozet = sefer_ozet.merge(sef_oz_arac_sefer_unq[['Sefer Numarası','maliyet']], on = 'Sefer Numarası', how = 'left')

In [292]:
sefer_ozet['oran'] = sefer_ozet['maliyet'] / sefer_ozet['capacities']

In [293]:
sef_oz_arac_sefer_unq['oran'].mean()

KeyError: 'oran'

In [ ]:
sef_oz_arac_sefer_unq['capacities'].unique()

In [ ]:
output4

In [ ]:
output5 = output4.drop_duplicates('sefer_id', keep = 'first')

In [ ]:
output5.to_excel('rotalar_17_aralık_model.xlsx', index = False)

In [ ]:
output4.groupby('sefer_id')['palet_list'].sum().reset_index()

In [ ]:
output4